In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns=None
pd.options.display.max_colwidth=300
from lower_case_dict import d_dict as D_dict
from lower_case_dict import d_describe as D_describe
from lower_case_dict import D_ambito_to_rama
import traduce as tr
import xlrd

Los **diccionarios de las variables originales** en la encuesta y los definidos para las **variables nuevas** (creadas inicialmente para el dataset de graduados 2019) se importan del **módulo lower_case_dict**:  

- **D_dict**; diccionarios de traducción de las variables categóricas. Construidos de acuerdo con los códigos facilitados en el archivo de diseño de la encuesta 'dr_EILU_GRAD_2019.xlsx' y con los definidos para las variables nuevas creadas en la inspeccion de la encuesta de graduados 2019. Para cada variable: D_dict[variable][código_categórico]=interpretación_código. 
- **D_describe**; diccionario de interpretación de las variables originales y nuevas: D_describe[variable]=interpretación_variable.   

También se importa del módulo lower_case_dict el diccionario D_ambito_to_rama que asocia los ámbitos de estudios a su rama de conocimiento. 

Se importa el **módulo traduce** y se transforman sus **funciones traduce() y leyenda()** para simplificar sus argumentos:


In [2]:
def traduce(col,check=True,save=True):
    return tr.traduce(df=df,dicc=D_dict,LO_=v_originales_L,LM=v_modelos,LM_=v_modelos_L,
                      col=col,check=check,save=save)
def leyenda(col):
    return tr.leyenda(D_dict,col)

# Recuperación del dataset
Dataset cargado en el DataFrame df_raw: todas las columnas aparecen en un único string.  
Las instrucciones para separar el string de datos en columnas están recogidas en el fichero disreg_recurciencia.xls. Las columnas relevantes para asignar nombres a las variables y establecer los caracteres correspondientes se cargan en el Dataframe df_aux: 
- Columna 'Nemotécnico' (renombrada 'Col_name'): denominación de las variables o columnas. 
- Columna 'Fin': último caracter correspondiente a cada columna. 

In [3]:
# Dataset con un único string sin separadores: 
df_raw=pd.read_csv('/home/bea/Escritorio/KSchool_TFM/Principales/Encuesta_RRHH_Ciencia_Tecnologia_2009/RRHH09.txt',
                   header=0,names=['string'])
df_raw.head()

,string
0,01000416196516EU271EU27NNNNNNNN21 0 2 07 8 122222211612NN 0NN 013031111992 9 311EU27198810 . .1418147200016EU271226 1 1 33311111432111110 NNN . .0 . . . NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . . ...
1,01000616197213EU271EU27NNNNNNNN21 0 0 08 7 151N33111612NN 0NN 01104110200011 211EU27199511 0 02 . NN .NNNNNN NNN NNN . . . . . 111111 1 1 NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . . ...
2,01000716196816EU271EU27NNNNNNNN21 1 1 02 2 155N11111612NN 0NN 015041102001 6 9 2EU27199161 1 11405185200517EU27223124 1 22122111212111110 NNN . .0 . . . NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . . ...
3,01000816196916EU271EU27NNNNNNNN26 0 2 03 3 127912111612NN 0NN 012070112002 2 8 5EU27199361 0 11376085199616EU2732314311 22112111112111110 NNN . .0 . . . EU2720062009 48EU2720052005 7EU2720002005 65NMOV . . .NMOV . . .NMOV . . .NMOV . . . 1 ...
4,01000916197516EU271EU27NNNNNNNN21 0 0 04 3 157N22211612NN 0NN 013031002002 6 4 0EU27199810 . .1408021200216EU271226 211 33222222211222220 NNN . .0 . . . NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . .NMOV . . . ...


In [4]:
# Dataset auxiliar: permite separar el string de datos en df_raw en columnas:
df_aux=pd.read_excel('/home/bea/Escritorio/KSchool_TFM/Principales/Encuesta_RRHH_Ciencia_Tecnologia_2009/disreg_recurciencia.xls',
                    sheet_name='RRHH09',skiprows=2,usecols=['Nemotécnico','Fin'],skipfooter=6)
# Columna de nombres de las variables de df_raw:
df_aux.rename(columns={'Nemotécnico':'Col_name'},inplace=True) 
# Columna de último caracter correspondiente a cada variable (numerados desde 1):
df_aux['Fin']=df_aux['Fin'].astype('int') 

In [5]:
# Denominaciones de columnas ('Col_name') y último caracter que ocupan ('Fin'):
df_aux.head()

,Col_name,Fin
0,MUIDENT,6
1,CCAARESI,8
2,ANONAC,12
3,CCAANAC,14
4,CONTNACIM,18


In [6]:
df_aux.tail()

,Col_name,Fin
194,COMPA,356
195,TUTOR,357
196,COOPERA,358
197,INVFUT,359
198,FACTOR,370


## Recuperación de las variables
Cada fila de df_aux corresponde a una variable (o columna) del dataset: incluye nombre de la variable y posición del último caracter. 
Para separar el string único df_raw['string'] en columnas y asignarle a cada una su denominacion original: 

In [7]:
# Bucle para separar el string único de df_raw en columnas de acuerdo con instrucciones en df_aux: 
inicio=0
for i in range(0,199):
    df_raw[df_aux['Col_name'].iloc[i]]=df_raw['string'].str[inicio:df_aux['Fin'].iloc[i]]
    inicio=df_aux['Fin'].iloc[i]
del df_raw['string'] 
del df_aux
df_raw.head()

,MUIDENT,CCAARESI,ANONAC,CCAANAC,CONTNACIM,RELA,CONTNAC1,CONTNAC2,CONTNAC3,SEXO,ESTADOCIVIL,DEPEN5,DEPEN18,DEPENMAS,NIVESTPA,NIVESTMA,NIVPROFPA,NIVPROFMA,ACTIPRIPA1,ACTIPRIMA1,EDUPRIM,EDUSECUN,BACHILLER,RES,CARES1,MESRES1,CARES2,MESRES2,CARES3,MESRES3,DOCTOR,CODIDOC,INFUN,INAPL,DESTEC,ANODOC,MESDOC,DURDOCANO,DURDOCMES,CONTEST,ANOEST,FINADOC,TRABDOC,ANOSINTRA,MESSINTRA,SITLAB,HTRAB,INGRESOS,AUTON,CNAE2,ANOINIALP,CAEMP,CONTEMP,SECTOR,CODALP,CATEGPROF,HLECT,TIPOCONT,JORLAB,BUSCATRAB,NIVELMIN,NIVELEST,RELTRABDOC,NIVELSAT1,NIVELSAT2,NIVELSAT3,NIVELSAT4,NIVELSAT5,NIVELSAT6,NIVELSAT7,NIVELSAT8,NIVELSAT9,NIVELSAT10,NIVELSAT11,NIVELSAT12,NIVELSAT13,ACTIANT,SECTORANT,ACTIPRIANT,ACTIANTANO,ACTIANTMES,TRABPOS,MOTIVOPOS1,MOTIVOPOS2,MOTIVOPOS3,MOTIVOPOS4,MOTIVOPOS5,MOTIVOPOS6,MOTIVOPOS7,MOTIVOPOS8,PORCINVES,PORCDOC,PORCOTRAS,FINANPOS,GESBUSEMP,PASOSEMP1,PASOSEMP2,PASOSEMP3,PASOSEMP4,PASOSEMP5,PASOSEMP6,PASOSEMP7,PASOSEMP8,PASOSEMP9,PASOSEMP10,PASOSEMP11,NOPASOSEMP1,NOPASOSEMP2,NOPASOSEMP3,NOPASOSEMP4,NOPASOSEMP5,NOPASOSEMP6,NOPASOSEMP7,NOPASOSEMP8,NOPASOSEMP9,CONT1,ANOIPAIS1,ANOFPAIS1,MESES1,CONT2,ANOIPAIS2,ANOFPAIS2,MESES2,CONT3,ANOIPAIS3,ANOFPAIS3,MESES3,CONT4,ANOIPAIS4,ANOFPAIS4,MESES4,CONT5,ANOIPAIS5,ANOFPAIS5,MESES5,CONT6,ANOIPAIS6,ANOFPAIS6,MESES6,CONT7,ANOIPAIS7,ANOFPAIS7,MESES7,MOTFUERA1,MOTFUERA2,MOTFUERA3,MOTFUERA4,MOTFUERA5,MOTFUERA6,MOTFUERA7,MOTVENIR1,MOTVENIR2,MOTVENIR3,MOTVENIR4,MOTVENIR5,MOTVENIR6,MOTVENIR7,PREVIRSE,MESANTESIR,MOTIRSE1,MOTIRSE2,MOTIRSE3,MOTIRSE4,MOTIRSE5,MOTIRSE6,CONTOUT,INV,MOTNOINV1,MOTNOINV2,MOTNOINV3,MOTNOINV4,MOTNOINV5,MOTNOINV6,MOTNOINV7,MOTNOINV8,MOTNOINV9,ALGINV,MOTINV1,MOTINV2,MOTINV3,MOTINV4,MOTINV5,MOTINV6,MOTINV7,MOTINV8,MOTINV9,MESESINV,ARTIC,LIBROS,NUMPAT,PATNAC,PATEU,PATCOOP,PATCOM,COMPA,TUTOR,COOPERA,INVFUT,FACTOR
0,010004,16,1965,16,EU27,1,EU27,NNNN,NNNN,2,1,0,2,0,7,8,1,2,2,2,2,2,2,1,16,12,NN,0,NN,0,1,303,1,1,1,1992,9,3,11,EU27,1988,1,0,.,.,1,41,8,1,47,2000,16,EU27,1,226,,1,,1,,3,3,3,1,1,1,1,1,4,3,2,1,1,1,1,1,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,0,,,,,,,,,1,0,,,,,,,,,,.,.,.,.,.,.,.,0,0,0,0,0,13.04720815
1,010006,16,1972,13,EU27,1,EU27,NNNN,NNNN,2,1,0,0,0,8,7,1,5,1,N,3,3,1,1,16,12,NN,0,NN,0,1,104,1,1,0,2000,11,2,11,EU27,1995,1,1,0,0,2,.,,,NN,.,NN,NNNN,,NNN,,,,,,,,,,,,,,,,,,,,,,,,NNN,.,.,,,,,,,,,,.,.,.,,1,1,1,1,1,1,,1,,1,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,0,,1,,,,,,,,1,1,,,,,,,,,36,0,0,0,0,0,0,0,0,0,1,0,13.04720815
2,010007,16,1968,16,EU27,1,EU27,NNNN,NNNN,2,1,1,1,0,2,2,1,5,5,N,1,1,1,1,16,12,NN,0,NN,0,1,504,1,1,0,2001,6,9,2,EU27,1991,6,1,1,1,1,40,5,1,85,2005,17,EU27,2,231,2,4,,1,,2,2,1,2,2,1,1,1,2,1,2,1,1,1,1,1,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,1,,,,,,,,,40,4,2,0,0,0,0,0,0,0,1,1,13.04720815
3,010008,16,1969,16,EU27,1,EU27,NNNN,NNNN,2,6,0,2,0,3,3,1,2,7,9,1,2,1,1,16,12,NN,0,NN,0,1,207,0,1,1,2002,2,8,5,EU27,1993,6,1,0,1,1,37,6,0,85,1996,16,EU27,3,231,4,3,1,1,,2,2,1,1,2,1,1,1,1,1,2,1,1,1,1,1,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,EU27,2006,2009,48,EU27,2005,2005,7,EU27,2000,2005,65,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,1,,,,,1,,,,,,2,5,,,1,,,,EUCA,1,,,,,,,,,,,,,,,,,,1,,120,3,10,0,0,0,0,0,0,0,1,1,13.04720815
4,010009,16,1975,16,EU27,1,EU27,NNNN,NNNN,2,1,0,0,0,4,3,1,5,7,N,2,2,2,1,16,12,NN,0,NN,0,1,303,1,0,0,2002,6,4,0,EU27,1998,1,0,.,.,1,40,8,0,21,2002,16,EU27,1,226,,2,1,1,,3,3,2,2,2,2,2,2,2,1,1,2,2,2,2,2,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,0,,,,,,,,,1,0,,,,,,,,,,.,.,.,.,.,.,.,0,0,0,0,0,13.04720815


In [8]:
df_raw.tail()

,MUIDENT,CCAARESI,ANONAC,CCAANAC,CONTNACIM,RELA,CONTNAC1,CONTNAC2,CONTNAC3,SEXO,ESTADOCIVIL,DEPEN5,DEPEN18,DEPENMAS,NIVESTPA,NIVESTMA,NIVPROFPA,NIVPROFMA,ACTIPRIPA1,ACTIPRIMA1,EDUPRIM,EDUSECUN,BACHILLER,RES,CARES1,MESRES1,CARES2,MESRES2,CARES3,MESRES3,DOCTOR,CODIDOC,INFUN,INAPL,DESTEC,ANODOC,MESDOC,DURDOCANO,DURDOCMES,CONTEST,ANOEST,FINADOC,TRABDOC,ANOSINTRA,MESSINTRA,SITLAB,HTRAB,INGRESOS,AUTON,CNAE2,ANOINIALP,CAEMP,CONTEMP,SECTOR,CODALP,CATEGPROF,HLECT,TIPOCONT,JORLAB,BUSCATRAB,NIVELMIN,NIVELEST,RELTRABDOC,NIVELSAT1,NIVELSAT2,NIVELSAT3,NIVELSAT4,NIVELSAT5,NIVELSAT6,NIVELSAT7,NIVELSAT8,NIVELSAT9,NIVELSAT10,NIVELSAT11,NIVELSAT12,NIVELSAT13,ACTIANT,SECTORANT,ACTIPRIANT,ACTIANTANO,ACTIANTMES,TRABPOS,MOTIVOPOS1,MOTIVOPOS2,MOTIVOPOS3,MOTIVOPOS4,MOTIVOPOS5,MOTIVOPOS6,MOTIVOPOS7,MOTIVOPOS8,PORCINVES,PORCDOC,PORCOTRAS,FINANPOS,GESBUSEMP,PASOSEMP1,PASOSEMP2,PASOSEMP3,PASOSEMP4,PASOSEMP5,PASOSEMP6,PASOSEMP7,PASOSEMP8,PASOSEMP9,PASOSEMP10,PASOSEMP11,NOPASOSEMP1,NOPASOSEMP2,NOPASOSEMP3,NOPASOSEMP4,NOPASOSEMP5,NOPASOSEMP6,NOPASOSEMP7,NOPASOSEMP8,NOPASOSEMP9,CONT1,ANOIPAIS1,ANOFPAIS1,MESES1,CONT2,ANOIPAIS2,ANOFPAIS2,MESES2,CONT3,ANOIPAIS3,ANOFPAIS3,MESES3,CONT4,ANOIPAIS4,ANOFPAIS4,MESES4,CONT5,ANOIPAIS5,ANOFPAIS5,MESES5,CONT6,ANOIPAIS6,ANOFPAIS6,MESES6,CONT7,ANOIPAIS7,ANOFPAIS7,MESES7,MOTFUERA1,MOTFUERA2,MOTFUERA3,MOTFUERA4,MOTFUERA5,MOTFUERA6,MOTFUERA7,MOTVENIR1,MOTVENIR2,MOTVENIR3,MOTVENIR4,MOTVENIR5,MOTVENIR6,MOTVENIR7,PREVIRSE,MESANTESIR,MOTIRSE1,MOTIRSE2,MOTIRSE3,MOTIRSE4,MOTIRSE5,MOTIRSE6,CONTOUT,INV,MOTNOINV1,MOTNOINV2,MOTNOINV3,MOTNOINV4,MOTNOINV5,MOTNOINV6,MOTNOINV7,MOTNOINV8,MOTNOINV9,ALGINV,MOTINV1,MOTINV2,MOTINV3,MOTINV4,MOTINV5,MOTINV6,MOTINV7,MOTINV8,MOTINV9,MESESINV,ARTIC,LIBROS,NUMPAT,PATNAC,PATEU,PATCOOP,PATCOM,COMPA,TUTOR,COOPERA,INVFUT,FACTOR
4117,520044,18,1949,01,EU27,1,EU27,NNNN,NNNN,1,4,0,0,1,3,3,2,5,0,N,1,1,1,1,01,12,NN,0,NN,0,1,602,0,1,0,2000,10,5,2,EU27,1994,6,1,3,2,3,.,,,NN,.,NN,NNNN,,NNN,,,,,,,,,,,,,,,,,,,,,,,,NNN,.,.,,,,,,,,,,.,.,.,,0,,,,,,,,,,,,,,,,1,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,,,,,,,,1,,72,0,1,0,0,0,0,0,0,1,1,1,2.557220801
4118,520046,18,1973,01,EU27,1,EU27,NNNN,NNNN,1,1,0,2,0,7,6,1,1,2,2,3,3,3,1,18,9,01,3,NN,0,1,601,1,1,1,2003,9,7,1,EU27,1996,8,1,3,3,1,30,3,0,85,2003,18,EU27,2,233,,5,1,1,,3,7,1,1,1,1,1,2,4,1,1,2,1,2,2,2,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,2,8,,,1,,,,EUCA,0,,1,,1,,,,,,1,,,,,,,,1,,50,10,3,1,1,0,0,0,0,0,1,1,2.557220801
4119,520047,13,1958,18,EU27,1,EU27,NNNN,NNNN,1,6,0,0,0,6,6,2,2,4,4,2,2,2,1,13,12,NN,0,NN,0,1,601,1,1,0,1994,9,2,9,EU27,1990,8,1,1,2,1,40,2,0,85,2009,13,EU27,3,231,3,4,2,1,,3,3,2,2,2,2,2,2,2,2,2,1,2,2,1,2,1,4,262,10,1,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,,,,,,,,1,,240,8,3,0,0,0,0,0,0,0,1,1,2.557220801
4120,520048,18,1967,10,EU27,1,EU27,NNNN,NNNN,1,1,0,1,1,6,2,2,5,2,N,3,1,1,1,18,12,NN,0,NN,0,1,503,1,0,0,2002,2,6,0,EU27,1994,8,1,0,0,1,70,8,0,85,1995,01,EU27,3,231,2,3,1,1,,2,2,2,1,1,2,3,3,2,1,1,1,1,1,1,2,0,,NNN,.,.,1,0,0,1,1,1,0,1,0,25,25,50,3,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,2,4,,,1,,,,EUCA,1,,,,,,,,,,,1,1,1,1,1,1,,,,192,50,60,1,0,1,0,1,0,1,1,1,2.557220801
4121,520050,18,1967,13,EU27,1,EU27,NNNN,NNNN,1,6,0,0,0,6,2,2,5,9,N,1,1,1,1,18,12,NN,0,NN,0,1,602,1,0,0,2004,1,8,0,EU27,1996,8,0,.,.,1,35,5,0,85,1998,18,EU27,2,233,,5,2,1,,3,3,3,2,2,2,1,1,3,3,2,2,2,2,2,2,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,0,,1,,1,1,,1,,1,0,,,,,,,,,,.,.,.,.,.,.,.,0,0,0,0,0,2.557220801


In [9]:
df_raw.shape

(4122, 199)

In [10]:
# Raw dataset en formato csv: 
df_raw.to_csv('/home/bea/Escritorio/KSchool_TFM/Principales/Encuesta_RRHH_Ciencia_Tecnologia_2009/raw_doct.csv')

In [11]:
df=df_raw.copy()

# Inspección y ajustes

Descripción de las variables de acuerdo con la información proporcionada en:

- El fichero de instrucciones y definiciones, disreg_recurciencia.xls
- La encuesta, en el fichero t1430225_09_cues.pdf

**Objetivo**: Inspección del dataset, imputación de valores perdidos debido a los itinerarios de la encuesta y creación de variables nuevas para análisis descriptivo y predictivo.   

De esta etapa del proceso se obtienen **dos datasets**:    

1- Un dataset con las **variables originales** (designadas con nombres en mayúsculas) a las que, a lo sumo, se aplica alguna de las transformaciones siguientes:  
- Imputación de valores perdidos que pueden deducirse de los itinerarios de la encuesta.  
- Sustitución de las no respuestas y las respuestas NS/NC por el valor None.  

2- Un **dataset nuevo** formado por algunas de las variables originales escogidas como candidatas a formar parte de los modelos descriptivos o predictivos y por algunas variables nuevas creadas a partir de las originales. Las variables de este dataset se designan por nombres en minúsculas para facilitar su identificación. Algunas de las transformaciones que dan lugar a este conjunto de variables son:  
- Sustitución del valor asociado a la respuesta No en las preguntas de respuesta Sí o No: se cambia el valor 2 asociado al No en el dataset original por el valor 0.  
- Creación de variables binarias para cada una de las categorías de algunas de las variables cualitativas del dataset original.  
- Creación de variables nuevas para información que puede deducirse de una o varias de las columnas del dataset original.  
- Creación de variables con información del dataset (año de la encuesta, población objeto de muestreo, etc.) con el fin de que sea posible identificar la procedencia de cada observación del el dataset que resulte de reunir todos los empleados en este estudio.  

En esta etapa **no se imputan valores a las no respuestas o a las respuestas NS/NC** que no puedan deducirse de los itinerarios de la encuesta. Solo se sustituyen por el valor None. Se procede así para evitar que los valores imputados interfieran en los resultados del análisis descriptivo. 

In [12]:
# Lista de variables originales. Permite recuperar el dataset original después de imputar los valores perdidos ...
# ... como consecuencia de los itinerarios de la encuesta. 
v_originales=df.columns.tolist()
# Lista a la que se agregarán las variables candidatas a ser incluidas en el modelo predictivo
v_modelos=list()
# Listas de variables literales (interpretación de códigos de variables categóricas)
v_originales_L=list()
v_modelos_L=list()

In [13]:
def no_respuesta(col,blank=' ',nsnc='9',tipo=None):
    """ Sustituye respuestas en blanco y NS/NC por None y establece tipo 'float' si es posible'.
    """
    if tipo=='int':
        blank=None
        nsnc=int(nsnc)
    try: df[col].replace({blank:None,nsnc:None},inplace=True)
    except: 
        serie=df[col].astype('O') # Evita conflictos cuando se aplica más de una vez
        serie.replace({blank:None,str(nsnc):None},inplace=True)
    try:df[col]=df[col].astype('float')
    except:pass # Algunas variables categóricas tienen valores no numéricos (e.g. CNAE)
    try:traduce(col,check=False)
    except:pass # Para variables no categóricas (sin diccionario)
    print (df[col].sort_values().unique())
    return

In [14]:
def redefine(col,nuevo=True,cat=True,si_no=False):
    """ Crea una columna denominada 'nuevo' (por defecto, el nombre original en minúsculas) a partir 
    de la columna 'col', la añade a la lista de variables para los modelos descriptivos y predictivos. 
    Si 'cat' True, asocia la variable a su diccionario, traduce y conserva la variable traducida.
    Si si_no=True, modifica el valor asociado a respuesta No (0 en lugar de 2).  
    """
    if nuevo==True:
        nuevo=col.lower()
    df[nuevo]=df[col]
    D_describe[nuevo]=D_describe[col]
    if str(nuevo) not in v_modelos: 
        v_modelos.append(nuevo)
    if cat==True: 
        D_dict[nuevo]=D_dict[col]
        traduce(nuevo,check=False)
        if str(nuevo)+'_' not in v_modelos_L: 
            v_modelos_L.append(str(nuevo)+'_')
    if si_no==True: 
        df[nuevo].replace({2:0},inplace=True)
        D_dict[nuevo]=D_dict['si_no']
        traduce(nuevo)
    return

In [15]:
def a_lista(col):
    if col==col.lower() and col==col.strip('_') and str(col) not in v_modelos:
        v_modelos.append(col)
    return

In [16]:
def print_listas():
    print ('v_originales_L:','\t',v_originales_L)
    print ('v_modelos:','\t',v_modelos,'\nv_modelos_L:','\t',v_modelos_L)
    return

In [17]:
def reset(col_or_list):
    """ Argumentos: columna o lista de columnas.
    Elimina la variable col y su literal col_ de las listas de variables redefinidas o literales.
    Si col es una variable en el dataset original, reestablece sus valores iniciales. 
    Si el argumento es una lista de columnas, aplica reset a cada columna de la lista.
    """
    global v_originales_L, v_modelos, v_modelos_L
    if type(col_or_list)!=list:
        L=[col_or_list]
    else: 
        L=col_or_list
    for col in L:
        if col==col.upper() and str(col)+'_' in v_originales_L:
            v_originales_L.remove(str(col)+'_')
        if col==col.lower() and str(col) in v_modelos:
            v_modelos.remove(str(col))
        if col==col.lower() and str(col)+'_' in LM_:
            v_modelos_L.remove(str(col)+'_')
        try:
            df[col]=df_raw[col]
            del df[str(col)+'_']
        except:pass
    return

## Nueva variable: 'año'

Año en el que se realizó la encuesta.
Se añade para que las observaciones de este dataset sean identificables cuando se reunan varios datasets.

In [18]:
df['año']=2009
a_lista('año')

## Nueva variable: 'est_encuesta'

Estudios oficiales objeto de la encuesta (los que definen la población objeto de muestreo).
Se añade para que las observaciones de este dataset sean identificables cuando se reunan varios datasets

In [19]:
df['est_encuesta']='doctorado'
a_lista('est_encuesta')

## 'MUIDENT'
**Índice** único del entrevistado.

In [20]:
df['MUIDENT']=df['MUIDENT'].astype('int') # Sin valores no informados

## 'CCAARESI'
**CCAA de residencia** el 31 de diciembre de 2009: '01'=Andalucía, '02'=Aragón, '03'=Asturias (Principado de),
'04'=Balears (Illes), '05'=Canarias, '06'=Cantabria, '07'=Castilla y León, '08'=Castilla - La Mancha, '09'=Cataluña, '10'=Comunidad Valenciana, '11'=Extremadura, '12'=Galicia, '13'=Madrid (Comunidad de), '14'=Murcia (Región de), '15'=Navarra (Comunidad Foral de), '16'=País Vasco, '17'=Rioja (La), '18'=Ceuta y Melilla. 

In [21]:
df['CCAARESI']=df['CCAARESI'].astype('int') # Sin valores no informados
df['CCAARESI'].sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18])

## 'ANONAC'
**Año de nacimiento**.

In [22]:
df['ANONAC']=df['ANONAC'].astype('int') # Sin valores no informados
df['ANONAC'].sort_values().unique()

array([1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947, 1948, 1949, 1950,
       1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961,
       1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972,
       1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980])

## Nuevas variables: 'edad_años' y 'edad'   
**edad_años**: edad en el momento de la encuesta.  
**edad**: frupo de edad en el momento de la encuesta.  
Se agrupa en intervalos para hacerla comparable con la de graduados y titulados de máster universitario. 

In [23]:
df['edad_años']=df['año']-df['ANONAC'] 

In [24]:
df['edad_años'].sort_values().unique()

array([29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
       46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62,
       63, 64, 65, 66, 67, 68, 69])

In [25]:
df['edad']=pd.cut(df['edad_años'],[0,29,34,100],labels=[1,2,3])
traduce('edad',check=False)
df[['edad_años','edad','edad_']].groupby('edad_').agg(['unique']) # Comprobación

,edad_años,edad
,unique,unique
edad_,,
De 30 a 34 años,"[34, 32, 30, 33, 31]","[2] Categories (1, int64): [2]"
De 35 y más años,"[44, 37, 41, 40, 48, 42, 36, 35, 53, 39, 51, 43, 38, 52, 47, 50, 49, 54, 45, 46, 61, 55, 63, 69, 57, 62, 68, 66, 64, 59, 58, 60, 56, 67, 65]","[3] Categories (1, int64): [3]"
Menores de 30 años,[29],"[1] Categories (1, int64): [1]"


## 'CCAANAC'
**CCAA de nacimiento**: '01'=Andalucía, '02'=Aragón, '03'=Asturias (Principado de), '04'=Balears (Illes), '05'=Canarias, '06'=Cantabria, '07'=Castilla y León, '08'=Castilla - La Mancha, '09'=Cataluña, '10'=Comunidad Valenciana, '11'=Extremadura, '12'=Galicia, '13'=Madrid (Comunidad de), '14'=Murcia (Región de), '15'=Navarra (Comunidad Foral de), '16'=País Vasco, '17'=Rioja (La), '18'=Ceuta y Melilla. 

In [26]:
df['CCAANAC'].unique() # No informados como 'NN' (no nacidos en España)

array(['16', '13', 'NN', '12', '15', '08', '07', '14', '10', '09', '01',
       '11', '04', '02', '03', '06', '18', '05', '17'], dtype=object)

In [27]:
df['CCAANAC'].replace({'NN':None},inplace=True) 
df['CCAANAC'].sort_values().unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', None], dtype=object)

## 'CONTNACIM'
**Continente de nacimiento**: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NNNN'=Blanco.

In [28]:
df['CONTNACIM'].unique() # Sin no informados

array(['EU27', 'RAME', 'REUR', 'EUCA', 'ASIA', 'AFRI', 'OCEA'],
      dtype=object)

## Nueva variable: 'eu27'   
**eu27**: 1=Nacido en EU27, 0=No nacido en EU27

In [29]:
df['eu27']=0
df['eu27'].mask(df['CONTNACIM']=='EU27',1,inplace=True)
traduce('eu27',check=False)
df[['eu27','eu27_','CONTNACIM']].groupby('eu27_').agg(['unique']) # Comprobación

,eu27,CONTNACIM
,unique,unique
eu27_,,
Nacido en EU27,[1],[EU27]
No nacido en EU27,[0],"[RAME, REUR, EUCA, ASIA, AFRI, OCEA]"


## 'RELA'
**Relación** que le une **con España**: '1'=Español por nacimiento, '2'=Español por nacionalización, '3'=Residente permanente, '4'=Residente temporal, '5'=Refugiado.

In [30]:
df['RELA']=df['RELA'].astype('int') # Sin no informados
df['RELA'].unique()

array([1, 2, 3, 4])

## Nueva variable: 'nacionalidad'

**Nacionalidad española** u otra: 0=Nacionalidad no española, 1=Nacionalidad española.

In [31]:
df['nacionalidad']=0
df['nacionalidad'].mask((df['RELA']==1)|(df['RELA']==2),1,inplace=True)
traduce('nacionalidad',check=False)
df[['nacionalidad_','nacionalidad','RELA']].groupby('nacionalidad_').agg(['unique']) # Comprobación

,nacionalidad,RELA
,unique,unique
nacionalidad_,,
Nacionalidad española,[1],"[1, 2]"
Nacionalidad no española,[0],"[3, 4]"


## 'CONTNAC1'
**Continente** del pais de su **primera nacionalidad**: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NNNN'=Blanco.

In [32]:
df['CONTNAC1'].unique() # Sin no informados

array(['EU27', 'RAME', 'REUR', 'EUCA', 'AFRI', 'ASIA'], dtype=object)

## 'CONTNAC2'
**Continente** del pais de su **segunda nacionalidad**: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NNNN'=Blanco.

In [33]:
df['CONTNAC2'].unique() # No informados como 'NNNN' 

array(['NNNN', 'EU27', 'EUCA', 'RAME', 'AFRI', 'REUR'], dtype=object)

In [34]:
df['CONTNAC2'].replace({'NNNN':None},inplace=True) 
df['CONTNAC2'].unique()

array([None, 'EU27', 'EUCA', 'RAME', 'AFRI', 'REUR'], dtype=object)

## 'CONTNAC3'
**Continente** del pais de su **tercera nacionalidad**: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NNNN'=Blanco.

In [35]:
df['CONTNAC3'].unique() # No informados como 'NNNN' 

array(['NNNN', 'RAME'], dtype=object)

In [36]:
df['CONTNAC3'].replace({'NNNN':None},inplace=True) 
df['CONTNAC3'].unique()

array([None, 'RAME'], dtype=object)

## 'SEXO' (redefinida como 'sexo')
**Sexo**: '1'=Hombre, '2'='Mujer'

In [37]:
df['SEXO']=df['SEXO'].astype('int') # Sin no informados
df['SEXO'].unique() 

array([2, 1])

In [38]:
df['sexo']=0
df['sexo'].mask(df['SEXO']==2,1,inplace=True)
traduce('sexo',check=False)
df[['sexo_','sexo','SEXO']].groupby('sexo_').agg(['unique']) # Comprobación

,sexo,SEXO
,unique,unique
sexo_,,
Hombre,[0],[1]
Mujer,[1],[2]


## 'ESTADOCIVIL' (redefinida como 'e_civil')
**Estado civil**: '1'=Casado, '2'=Pareja de hecho, '3'=Separado, '4'=Divorciado, '5'=Viudo, '6'=Soltero.

Nota: se modifican los valores numéricos para hacerlos coincidir con empleados en las encuestas de graduados y titulados de máster universitario. Se omite la categoría 'Pareja de hecho' por la misma razón.

In [39]:
df['ESTADOCIVIL']=df['ESTADOCIVIL'].astype('int') # Sin no informados
df['ESTADOCIVIL'].sort_values().unique()

array([1, 2, 3, 4, 5, 6])

In [40]:
df['e_civil']=None
df['e_civil'].mask(df['ESTADOCIVIL']==6,1,inplace=True)
df['e_civil'].mask(df['ESTADOCIVIL']==1,2,inplace=True)
df['e_civil'].mask(df['ESTADOCIVIL']==5,3,inplace=True)
df['e_civil'].mask(df['ESTADOCIVIL']==3,4,inplace=True)
df['e_civil'].mask(df['ESTADOCIVIL']==4,5,inplace=True)
traduce('e_civil',check=False)
df[['e_civil_','e_civil','ESTADOCIVIL']].groupby('e_civil_').agg(['unique']) # Comprobación

,e_civil,ESTADOCIVIL
,unique,unique
e_civil_,,
Casado,[2],[1]
Divorciado,[5],[4]
Separado,[4],[3]
Soltero,[1],[6]
Viudo,[3],[5]


## 'DEPEN5'
Número de **personas dependientes a su cargo de 5 años o menos**.  

In [41]:
df['DEPEN5']=df['DEPEN5'].astype('int') # Sin no informados
df['DEPEN5'].sort_values().unique()

array([0, 1, 2, 3, 5])

## 'DEPEN18'
Número de **personas dependientes a su cargo de 6 a 18 años**.

In [42]:
df['DEPEN18']=df['DEPEN18'].astype('int') # Sin no informados
df['DEPEN18'].sort_values().unique()

array([0, 1, 2, 3, 4, 5, 6])

## 'DEPENMAS'
Número de **personas dependientes a su cargo de más de 18 años**.

In [43]:
df['DEPENMAS']=df['DEPENMAS'].astype('int') # Sin no informados
df['DEPENMAS'].sort_values().unique()

array([0, 1, 2, 3, 4, 6])

## 'NIVESTPA' (redefinida como 'estudios_pa')
**Nivel educativo del padre**: '1'=Sin estudios, '2'=Enseñanza Primaria Incompleta, '3'=ESO,EGB,Primaria completa o equivalente, '4'=FP I, '5'=FP II, '6'=Bachiller Superior,BUP,COU, '7'=Diplomado,Arquitecto o Ingeniero Técnico o equiv, '8'=Licenciado,Arquitecto o Ingeniero Superior o equiv, '9'=Doctorado,postgrado o especialización para licenciados, '10'=NS/NC.

Nota: se modifican algunos los valores en 'estudios_pa' para hacerlos consistentes con los definidos en las encuesta de graduados y titulados de máster universitario. 

In [44]:
df['NIVESTPA'].sort_values().unique()  # No informados como '10'

array(['1 ', '10', '2 ', '3 ', '4 ', '5 ', '6 ', '7 ', '8 ', '9 '],
      dtype=object)

In [45]:
no_respuesta('NIVESTPA',nsnc='10')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. nan]


In [46]:
df['estudios_pa']=df['NIVESTPA']
df['estudios_pa'].mask(df['NIVESTPA']==6,5,inplace=True)
df['estudios_pa'].mask(df['NIVESTPA']==4,6,inplace=True)
df['estudios_pa'].mask(df['NIVESTPA']==5,7,inplace=True)
df['estudios_pa'].mask(df['NIVESTPA']==7,8,inplace=True)
df['estudios_pa'].mask(df['NIVESTPA']==8,8,inplace=True)
df['estudios_pa'].mask(df['NIVESTPA']==9,8,inplace=True)
traduce('estudios_pa',check=False)
df[['estudios_pa_','estudios_pa','NIVESTPA']].groupby('estudios_pa_').agg(['unique'])

,estudios_pa,NIVESTPA
,unique,unique
estudios_pa_,,
Educación primaria completa,[3.0],[3.0]
Educación primaria incompleta (ha asistido menos de 5 años a la escuela),[2.0],[2.0]
Enseñanzas profesionales de grado medio o equivalentes,[6.0],[4.0]
Enseñanzas profesionales de grado superior o equivalentes,[7.0],[5.0]
"Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu)",[5.0],[6.0]
"Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes",[8.0],"[7.0, 8.0, 9.0]"
No sabe leer o escribir,[1.0],[1.0]


## Nueva variable: 'grado_med_pa'
**Padre** con o sin **estudios de enseñanzas medias o superiores**, incluidos los universitarios: 1=Con estudios de enseñanzas medias o superiores, 0=Sin estudios de enseñanzas medias o superiores.

In [47]:
df['grad_med_pa']=0
df['grad_med_pa'].mask(df['estudios_pa']>=5,1,inplace=True)
traduce('grad_med_pa')
df[['estudios_pa_','grad_med_pa']].groupby('grad_med_pa').agg(['unique']) # Comprobación: 

                                                  aux_traduce
                                                       unique
grad_med_pa                                                  
0            [Sin estudios de enseñanzas medias o superiores]
1            [Con estudios de enseñanzas medias o superiores]


,estudios_pa_
,unique
grad_med_pa,
0,"[Educación primaria incompleta (ha asistido menos de 5 años a la escuela), Educación primaria completa, No sabe leer o escribir, nan]"
1,"[Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes, Enseñanzas profesionales de grado medio o equivalentes, Enseñanzas profesionales de grado superior o equivalentes, Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu)]"


## Nueva variable: 'univ_pa'
**Padre** con o sin **estudios universitarios**: 1=Con estudios universitarios, 0=Sin estudios universitarios 

In [48]:
df['univ_pa']=0
df['univ_pa'].mask(df['estudios_pa']==8,1,inplace=True)
traduce('univ_pa')
df[['estudios_pa_','univ_pa']].groupby('univ_pa').agg(['unique']) # Comprobación: 

                           aux_traduce
                                unique
univ_pa                               
0        [Sin estudios universitarios]
1        [Con estudios universitarios]


,estudios_pa_
,unique
univ_pa,
0,"[Educación primaria incompleta (ha asistido menos de 5 años a la escuela), Educación primaria completa, Enseñanzas profesionales de grado medio o equivalentes, Enseñanzas profesionales de grado superior o equivalentes, No sabe leer o escribir, Estudios de Bachillerato (Bachillerato LOGSE, BUP, C..."
1,"[Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes]"


## 'NIVESTMA'  (redefinida como 'estudios_ma')
**Nivel educativo de la madre**: '1'=Sin estudios, '2'=Enseñanza Primaria Incompleta, '3'=ESO,EGB,Primaria completa o equivalente, '4'=FP I, '5'=FP II, '6'=Bachiller Superior,BUP,COU, '7'=Diplomado,Arquitecto o Ingeniero Técnico o equiv, '8'=Licenciado,Arquitecto o Ingeniero Superior o equiv, '9'=Doctorado,postgrado o especialización para licenciados, '10'=NS/NC.

Nota: se modifican algunos los valores en 'estudios_ma' para hacerlos consistentes con los definidos en las encuesta de graduados y titulados de máster universitario. 

In [49]:
df['NIVESTMA'].sort_values().unique()  # No informados como '10'

array(['1 ', '10', '2 ', '3 ', '4 ', '5 ', '6 ', '7 ', '8 ', '9 '],
      dtype=object)

In [50]:
no_respuesta('NIVESTMA',nsnc='10')

[ 1.  2.  3.  4.  5.  6.  7.  8.  9. nan]


In [51]:
df['estudios_ma']=df['NIVESTMA']
df['estudios_ma'].mask(df['NIVESTMA']==6,5,inplace=True)
df['estudios_ma'].mask(df['NIVESTMA']==4,6,inplace=True)
df['estudios_ma'].mask(df['NIVESTMA']==5,7,inplace=True)
df['estudios_ma'].mask(df['NIVESTMA']==7,8,inplace=True)
df['estudios_ma'].mask(df['NIVESTMA']==8,8,inplace=True)
df['estudios_ma'].mask(df['NIVESTMA']==9,8,inplace=True)
traduce('estudios_ma',check=False)
df[['estudios_ma_','estudios_ma','NIVESTMA']].groupby('estudios_ma_').agg(['unique'])

,estudios_ma,NIVESTMA
,unique,unique
estudios_ma_,,
Educación primaria completa,[3.0],[3.0]
Educación primaria incompleta (ha asistido menos de 5 años a la escuela),[2.0],[2.0]
Enseñanzas profesionales de grado medio o equivalentes,[6.0],[4.0]
Enseñanzas profesionales de grado superior o equivalentes,[7.0],[5.0]
"Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu)",[5.0],[6.0]
"Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes",[8.0],"[8.0, 7.0, 9.0]"
No sabe leer o escribir,[1.0],[1.0]


## Nueva variable: 'grado_med_ma'
**Madre** con o sin **estudios de enseñanzas medias o superiores**, incluidos los universitarios: 1=Con estudios de enseñanzas medias o superiores, 0=Sin estudios de enseñanzas medias o superiores.

In [52]:
df['grad_med_ma']=0
df['grad_med_ma'].mask(df['estudios_ma']>=5,1,inplace=True)
traduce('grad_med_ma')
df[['estudios_ma_','grad_med_ma']].groupby('grad_med_ma').agg(['unique']) # Comprobación: 

                                                  aux_traduce
                                                       unique
grad_med_ma                                                  
0            [Sin estudios de enseñanzas medias o superiores]
1            [Con estudios de enseñanzas medias o superiores]


,estudios_ma_
,unique
grad_med_ma,
0,"[Educación primaria incompleta (ha asistido menos de 5 años a la escuela), Educación primaria completa, No sabe leer o escribir, nan]"
1,"[Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes, Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu), Enseñanzas profesionales de grado superior o equivalentes, Enseñanzas profesionales de grado medio o equivalentes]"


## Nueva variable: 'univ_ma'
**Madre** con o sin **estudios universitarios**: 1=Con estudios universitarios, 0=Sin estudios universitarios 

In [53]:
df['univ_ma']=0
df['univ_ma'].mask(df['estudios_ma']==8,1,inplace=True)
traduce('univ_ma')
df[['estudios_ma_','univ_ma']].groupby('univ_ma').agg(['unique']) # Comprobación: 

                           aux_traduce
                                unique
univ_ma                               
0        [Sin estudios universitarios]
1        [Con estudios universitarios]


,estudios_ma_
,unique
univ_ma,
0,"[Educación primaria incompleta (ha asistido menos de 5 años a la escuela), Educación primaria completa, Estudios de Bachillerato (Bachillerato LOGSE, BUP, COU, Preu), No sabe leer o escribir, Enseñanzas profesionales de grado superior o equivalentes, Enseñanzas profesionales de grado medio o equ..."
1,"[Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes]"


## 'NIVPROFPA'
**Sector** al que pertenece la principal organización en la que **ha trabajado el padre** del encuestado: 1=Empresas, 2=AAPP, 3=Enseñanza superior, 4=IPSFL, 5=No ha trabajado nunca, 6=otros.

In [54]:
df['NIVPROFPA']=df['NIVPROFPA'].astype('int') # Sin no informados
df['NIVPROFPA'].sort_values().unique()

array([1, 2, 3, 4, 6])

## 'NIVPROFMA'
**Sector** al que pertenece la principal organización en la que **ha trabajado la madre** del encuestado: 1=Empresas, 2=AAPP, 3=Enseñanza superior, 4=IPSFL, 5=No ha trabajado nunca, 6=otros.

In [55]:
df['NIVPROFMA']=df['NIVPROFMA'].astype('int') # Sin no informados
df['NIVPROFMA'].sort_values().unique()

array([1, 2, 3, 4, 5, 6])

## 'ACTIPRIPA1'
**Sector de actividad laboral** principal del **padre**: Código de actividad laboral válido a 1 dígito o blanco

In [56]:
df['ACTIPRIPA1']=df['ACTIPRIPA1'].astype('int') # Sin no informados
df['ACTIPRIPA1'].sort_values().unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

## 'ACTIPRIMA1'
**Sector de actividad laboral** principal de la **madre**: Código de actividad laboral válido a 1 dígito o blanco

In [57]:
df['ACTIPRIMA1'].unique() # No informados como 'N'

array(['2', 'N', '9', '5', '1', '4', '7', '6', '3', '8', '0'],
      dtype=object)

In [58]:
df[['NIVPROFMA','ACTIPRIMA1']].groupby('NIVPROFMA').agg(['unique'])

,ACTIPRIMA1
,unique
NIVPROFMA,
1,"[5, 4, 2, 9, 7, 6, 3, 8, 1, 0]"
2,"[2, 9, 1, 4, 5, 3]"
3,"[2, 3, 1]"
4,"[2, 9, 4, 5]"
5,[N]
6,[2]


In [59]:
df['ACTIPRIMA1'].replace({'N':None},inplace=True) 
df['ACTIPRIMA1']=df['ACTIPRIMA1'].astype('float')  
df['ACTIPRIMA1'].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., nan])

## 'EDUPRIM'
**Tipo de centro** en que cursó la **educación primaria**: 1=Centro Público, 2=Centro Concertado, 3=Centro Privado.

In [60]:
df['EDUPRIM']=df['EDUPRIM'].astype('int') # Sin no informados
df['EDUPRIM'].sort_values().unique()

array([1, 2, 3])

## 'EDUSECUN'
**Tipo de centro** en que cursó la **educacion secundaria o la FP II**: 1=Centro Público, 2=Centro Concertado, 3=Centro Privado.

In [61]:
df['EDUSECUN']=df['EDUSECUN'].astype('int') # Sin no informados
df['EDUSECUN'].sort_values().unique()

array([1, 2, 3])

## 'BACHILLER'
**Tipo de centro** en que cursó el **bachillerato**: 1=Centro Público, 2=Centro Concertado, 3=Centro Privado.

In [62]:
df['BACHILLER']=df['BACHILLER'].astype('int') # Sin no informados
df['BACHILLER'].sort_values().unique()

array([1, 2, 3])

### Inspección de cambios de centro en bachillerato
El bachillerato no tiene concierto (es educación secundaria no obligatoria). Se estudian los cambios de centro al acceder a bachillerato para valorar si está justificado tomar el bachillerato como nivel de referencia para resumir el tipo de centro -público o privado- en una única columna. 
Se toma esta decisión por tres razones: 
- El resultado académco en bachillerato tiene influencia directa en el acceso a la universidad.  
- No está subvencionado en los centros concertados.  
- La el cambio de centro más frecuente entre ESO y bachillerato es de concertadas a pública (refleja el aspecto económico de la elección).

In [63]:
df['bach_aux']=df['BACHILLER']
preuniv=['EDUPRIM','EDUSECUN','BACHILLER','bach_aux']

In [64]:
df_preuniv=df[preuniv].pivot_table(values='bach_aux',index=['EDUPRIM','EDUSECUN'],columns='BACHILLER',
                        aggfunc=['count'],margins=True,fill_value=0)
df_preuniv#.rank(axis=1,ascending=False)
# Los cambios de centro más frecuentes entre secundaria y bachillerato son hacia la pública (a pesar de que lo ...
# ... más frecuente es permanecer en el mismo tipo de centro)

count                 
BACHILLER            1    2     3   All
EDUPRIM EDUSECUN                       
1       1         1508   33    47  1588
        2           22   71     5    98
        3           21    2   105   128
2       1          302    2     2   306
        2          156  514    80   750
        3            4    1    36    41
3       1          192    0     1   193
        2            9   31     5    45
        3          145   12   816   973
All               2359  666  1097  4122

In [65]:
# Para calcular porcentajes para cada fila: nuevas columnas de porcentajes por filas
for i in range(3):
    df_preuniv['prop'+str(i+1)]=df_preuniv[df_preuniv.columns[i]]/df_preuniv[df_preuniv.columns[3]]
df_preuniv['check']=df_preuniv[['prop'+str(i+1) for i in range(3)]].sum(axis=1) # Comprobación
df_preuniv

count                      prop1     prop2     prop3 check
BACHILLER            1    2     3   All                                    
EDUPRIM EDUSECUN                                                           
1       1         1508   33    47  1588  0.949622  0.020781  0.029597   1.0
        2           22   71     5    98  0.224490  0.724490  0.051020   1.0
        3           21    2   105   128  0.164062  0.015625  0.820312   1.0
2       1          302    2     2   306  0.986928  0.006536  0.006536   1.0
        2          156  514    80   750  0.208000  0.685333  0.106667   1.0
        3            4    1    36    41  0.097561  0.024390  0.878049   1.0
3       1          192    0     1   193  0.994819  0.000000  0.005181   1.0
        2            9   31     5    45  0.200000  0.688889  0.111111   1.0
        3          145   12   816   973  0.149024  0.012333  0.838643   1.0
All               2359  666  1097  4122  0.572295  0.161572  0.266133   1.0

In [66]:
del df_preuniv

In [67]:
# Tipo de centro público o privado definido en referencia al bachillerato (único nivel educativo no financiado ...
# ... por concierto de los tres sobre los que se pregunta)
df['tipo_centro']=0
df['tipo_centro'].mask(df['BACHILLER']==1,1,inplace=True) 
traduce('tipo_centro')
df[['tipo_centro_','BACHILLER']].groupby('tipo_centro_',dropna=False).agg(['unique']) #Comprobación

            aux_traduce
                 unique
tipo_centro            
0             [Privado]
1             [Público]


,BACHILLER
,unique
tipo_centro_,
Privado,"[2, 3]"
Público,[1]


## 'RES'
¿**Residió en España** en algún periodo de 2009? 0=No, 1=Sí. 

In [68]:
df['RES']=df['RES'].astype('int') # Sin no informados
df['RES'].unique()

array([1, 0])

In [69]:
df['RES'].mean()

0.9798641436196022

## 'CARES1'
**Primera CCAA de residencia en 2009**: '01'=Andalucía, '02'=Aragón, '03'=Asturias (Principado de), '04'=Balears (Illes), '05'=Canarias, '06'=Cantabria, '07'=Castilla y León, '08'=Castilla - La Mancha, '09'=Cataluña, '10'=Comunidad Valenciana, '11'=Extremadura, '12'=Galicia, '13'=Madrid (Comunidad de), '14'=Murcia (Región de), '15'=Navarra (Comunidad Foral de), '16'=País Vasco, '17'=Rioja (La), '18'=Ceuta y Melilla. 

In [70]:
df['CARES1'].unique() # No informados como 'NN' 

array(['16', 'NN', '13', '09', '08', '01', '07', '10', '11', '04', '05',
       '06', '12', '03', '14', '02', '17', '15', '18'], dtype=object)

In [71]:
df[['RES','CARES1']].groupby('RES').agg(['unique']) 

,CARES1
,unique
RES,
0,[NN]
1,"[16, 13, 09, 08, 01, 07, 10, 11, 04, 05, 06, 12, 03, 14, 02, 17, 15, 18]"


In [72]:
df['CARES1'].replace({'NN':None},inplace=True) 
df['CARES1'].sort_values().unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '14', '15', '16', '17', '18', None], dtype=object)

## 'MESRES1'
**Duración en meses** del periodo de 2009 en que **residió en la primera CCAA** identificada.

In [73]:
df['MESRES1'].unique() # No informados como  ' .' 

array(['12', ' .', ' 8', '10', ' 6', ' 9', '11', ' 7', ' 5', ' 3', ' 4',
       ' 1', ' 2'], dtype=object)

In [74]:
df[['RES','MESRES1']].groupby('RES').agg(['unique']) 

,MESRES1
,unique
RES,
0,[ .]
1,"[12, 8, 10, 6, 9, 11, 7, 5, 3, 4, 1, 2]"


In [75]:
df['MESRES1'].replace({' .':None},inplace=True) 
df['MESRES1']=df['MESRES1'].astype('float')
df['MESRES1'].sort_values().unique()

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., nan])

In [76]:
df[['RES','MESRES1']].groupby('RES').agg(['unique'])

,MESRES1
,unique
RES,
0,[nan]
1,"[12.0, 8.0, 10.0, 6.0, 9.0, 11.0, 7.0, 5.0, 3.0, 4.0, 1.0, 2.0]"


## 'CARES2'
**Segunda CCAA de residencia** en 2009: '01'=Andalucía, '02'=Aragón, '03'=Asturias (Principado de), '04'=Balears (Illes), '05'=Canarias, '06'=Cantabria, '07'=Castilla y León, '08'=Castilla - La Mancha, '09'=Cataluña, '10'=Comunidad Valenciana, '11'=Extremadura, '12'=Galicia, '13'=Madrid (Comunidad de), '14'=Murcia (Región de), '15'=Navarra (Comunidad Foral de), '16'=País Vasco, '17'=Rioja (La), '18'=Ceuta y Melilla. 

In [77]:
df['CARES2'].unique() # No informados como 'NN' 

array(['NN', '08', '10', '13', '01', '09', '11', '04', '03', '02', '05',
       '16', '12', '07', '17', '06', '15', '18'], dtype=object)

In [78]:
df['CARES2'].replace({'NN':None},inplace=True) # None when not lived in two different CCAA during 2009
df['CARES2'].sort_values().unique()

array(['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11',
       '12', '13', '15', '16', '17', '18', None], dtype=object)

## 'MESRES2'
**Duración en meses** del periodo de 2009 en que **residió en la segunda CCAA** identificada.

In [79]:
df['MESRES2'].unique() # No informados como ' .' o '0'

array([' 0', ' .', ' 6', ' 4', ' 1', ' 2', ' 3', ' 7', ' 9', ' 5', '10',
       '11', ' 8'], dtype=object)

In [80]:
df[['CARES2','MESRES2']].groupby('CARES2',dropna=False).agg(['unique']) 

,MESRES2
,unique
CARES2,
01,"[ 4, 1, 5, 6, 2, 3, 7, 8, 10]"
02,"[ 4, 6, 3, 10, 1, 8, 11]"
03,"[ 4, 1, 2, 3, 6]"
04,"[ 1, 6, 9, 3]"
05,"[ 2, 3, 9, 1]"
06,"[ 2, 10, 1, 3, 5]"
07,"[ 1, 2, 9, 5, 6, 4]"
08,"[ 6, 7, 3, 2, 1, 4]"


In [81]:
df['MESRES2'].replace({' .':0},inplace=True) 
df['MESRES2']=df['MESRES2'].astype('int')
df['MESRES2'].sort_values().unique() 
# CCAA no ordenadas de acuerdo con la duración de la residencia (algunos valores de 'MESRES2' mayores que 6)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [82]:
df[['CARES2','MESRES2']].groupby('CARES2',dropna=False).agg(['unique'])

,MESRES2
,unique
CARES2,
01,"[4, 1, 5, 6, 2, 3, 7, 8, 10]"
02,"[4, 6, 3, 10, 1, 8, 11]"
03,"[4, 1, 2, 3, 6]"
04,"[1, 6, 9, 3]"
05,"[2, 3, 9, 1]"
06,"[2, 10, 1, 3, 5]"
07,"[1, 2, 9, 5, 6, 4]"
08,"[6, 7, 3, 2, 1, 4]"


## 'CARES3'
**Tercera CCAA de residencia** en 2009: '01'=Andalucía, '02'=Aragón, '03'=Asturias (Principado de), '04'=Balears (Illes), '05'=Canarias, '06'=Cantabria, '07'=Castilla y León, '08'=Castilla - La Mancha, '09'=Cataluña, '10'=Comunidad Valenciana, '11'=Extremadura, '12'=Galicia, '13'=Madrid (Comunidad de), '14'=Murcia (Región de), '15'=Navarra (Comunidad Foral de), '16'=País Vasco, '17'=Rioja (La), '18'=Ceuta y Melilla. 

In [83]:
df['CARES3'].unique() # No informados como 'NN' 

array(['NN', '01', '12', '09', '03', '13', '07'], dtype=object)

In [84]:
df['CARES3'].replace({'NN':None},inplace=True) 
df['CARES3'].sort_values().unique() 

array(['01', '03', '07', '09', '12', '13', None], dtype=object)

## 'MESRES3'
**Duración en meses** del periodo de 2009 en que **residió en la tercera CCAA** identificada.

In [85]:
df['MESRES3'].unique() # No informados como ' .' o '0'

array([' 0', ' .', ' 2', ' 3', ' 1', '10'], dtype=object)

In [86]:
df[['CARES3','MESRES3']].groupby('CARES3',dropna=False).agg(['unique']) # Blank as ' .' or '0' when no answer is expected

,MESRES3
,unique
CARES3,
01,"[ 2, 1]"
03,"[ 3, 1]"
07,[ 1]
09,"[ 2, 1]"
12,"[ 3, 2]"
13,"[10, 2, 1]"
NaN,"[ 0, .]"


In [87]:
df['MESRES3'].replace({' .':0},inplace=True) # 0 when not lived in three different CCAA during 2009
df['MESRES3']=df['MESRES3'].astype('int')
df['MESRES3'].sort_values().unique() 

array([ 0,  1,  2,  3, 10])

In [88]:
df[['CARES3','MESRES3']].groupby('CARES3',dropna=False).agg(['unique'])

,MESRES3
,unique
CARES3,
01,"[2, 1]"
03,"[3, 1]"
07,[1]
09,"[2, 1]"
12,"[3, 2]"
13,"[10, 2, 1]"
NaN,[0]


## 'DOCTOR' (redefinida como 'doctor')
**Título de doctor anterior** al 31 de diciembre de 2009: 0=No, 1=Sí

In [89]:
df['DOCTOR']=df['DOCTOR'].astype('int') # Sin no informados 
df['DOCTOR'].unique()

array([1])

In [90]:
df['doctor']=df['DOCTOR']
traduce('doctor')

       aux_traduce
            unique
doctor            
1             [Sí]


## 'CODIDOC' (redefinida como 'titulo_doc')
Código de **estudios de doctorado**:  

CIENCIAS NATURALES: 101=Matemáticas, 102=Informática y tecnologías de la información, 103=Ciencias físicas, 104=Ciencias químicas, 105=Ciencias de la tierra y medio ambiente, 106=Biología (excluyendo agricultura y ciencias médicas), 107=Otras ciencias naturales  

INGENIERÍA Y TECNOLOGÍA: 201=Ingeniería civil, 202=Ingeniería eléctrica, electrónica y de telecomunicaciones, 203=Ingeniería mecánica, 204=Ingeniería química, 205=Ingeniería de materiales, 206=Ingeniería médica, 207=Ingeniería medioambiental, 208=Biotecnología medioambiental, 209=Biotecnología industrial, 210=Nanotecnología, 211=Otras ingenierías y tecnologías (comida, bebida y otras)  

CIENCIAS MÉDICAS: 301=Medicina básica, 302=Medicina clínica, 303=Ciencias de la salud, 304=Biotecnología médica, 305=Otras ciencias médicas (forenses y otras ciencias médicas)  

CIENCIAS DE LA AGRICULTURA: 401=Agricultura, ciencias forestales y piscifactorías, 402=Ciencias de los animales y de la leche, 403=Veterinaria, 404=Biotecnología agrícola, 405=Otras ciencias de la agricultura  

CIENCIAS SOCIALES: 501=Psicología, 502=Economía y empresas, 503=Ciencias de la educación, 504=Sociología, 505=Derecho, 506=Ciencias políticas, 507=Geografía económica y social, 508=Periodismo y comunicaciones, 509=Otras ciencias sociales  

HUMANIDADES: 601=Historia y arqueología, 602=Lenguaje y literatura, 603=Filosofía, ética y religión, 604=Arte (historia del arte, bellas artes y música), 605=Otras humanidades  

In [91]:
df['CODIDOC'].sort_values().unique() # Sin no informados

array(['101', '102', '103', '104', '105', '106', '107', '201', '202',
       '203', '204', '205', '206', '207', '208', '209', '210', '211',
       '301', '302', '303', '304', '305', '401', '402', '403', '404',
       '405', '501', '502', '503', '504', '505', '506', '507', '508',
       '509', '601', '602', '603', '604', '605'], dtype=object)

### Diccionario de códigos de estudios de doctorado
Se importa el diccionario para la columna 'CODIDOC' desde el fichero de diseño de la encuesta disreg_recurciencia.xls.

In [92]:
ruta_aux_codidoc='data/disreg_recurciencia.xls'
df_aux_codidoc=pd.read_excel(ruta_aux_codidoc,sheet_name='ANEXO II',usecols='A',header=1,skiprows=0)
df_aux_codidoc.head()

,Clasificación 1. Campos de Ciencia y Tecnología
0,CIENCIAS NATURALES
1,101 Matemáticas
2,102 Informática y tecnologías de la información
3,103 Ciencias físicas
4,104 Ciencias químicas


In [93]:
df_aux_codidoc.tail(7)

,Clasificación 1. Campos de Ciencia y Tecnología
41,509 Otras ciencias sociales
42,HUMANIDADES
43,601 Historia y arqueología
44,602 Lenguaje y literatura
45,"603 Filosofía, ética y religión"
46,"604 Arte (historia del arte, bellas artes y música)"
47,605 Otras humanidades


In [94]:
# Se eliminan los títulos de grupo (en mayúsculas)
len_0=len(df_aux_codidoc)
for i in range(len(df_aux_codidoc['Clasificación 1. Campos de Ciencia y Tecnología'])):
    if df_aux_codidoc['Clasificación 1. Campos de Ciencia y Tecnología'].iloc[i].upper()\
                ==df_aux_codidoc['Clasificación 1. Campos de Ciencia y Tecnología'].iloc[i]:
        df_aux_codidoc['Clasificación 1. Campos de Ciencia y Tecnología'].iloc[i]=None
df_aux_codidoc.dropna(inplace=True)
df_aux_codidoc.reset_index(drop=True,inplace=True)
len(df_aux_codidoc)-len_0 # Comprobación

-6

In [95]:
df_aux_codidoc.head()

,Clasificación 1. Campos de Ciencia y Tecnología
0,101 Matemáticas
1,102 Informática y tecnologías de la información
2,103 Ciencias físicas
3,104 Ciencias químicas
4,105 Ciencias de la tierra y medio ambiente


In [96]:
df_aux_codidoc['codigos']=df_aux_codidoc['Clasificación 1. Campos de Ciencia y Tecnología'].str.split(' ',1).str[0]
df_aux_codidoc['codigos'].head()

0    101
1    102
2    103
3    104
4    105
Name: codigos, dtype: object

In [97]:
df_aux_codidoc['titulos']=df_aux_codidoc['Clasificación 1. Campos de Ciencia y Tecnología'].str.split(' ',1).str[1]
df_aux_codidoc['titulos'].head()

0                                    Matemáticas
1    Informática y tecnologías de la información
2                               Ciencias físicas
3                              Ciencias químicas
4         Ciencias de la tierra y medio ambiente
Name: titulos, dtype: object

In [98]:
D_dict['CODIDOC']=dict()
for i in range(len(df_aux_codidoc['codigos'])):
    D_dict['CODIDOC'][df_aux_codidoc['codigos'].iloc[i]]=df_aux_codidoc['titulos'].iloc[i]
    D_dict['CODIDOC'][int(df_aux_codidoc['codigos'].iloc[i])]=df_aux_codidoc['titulos'].iloc[i]

In [99]:
D_describe['CODIDOC']='Código de estudios de doctorado'
traduce('CODIDOC')

                                                          aux_traduce
                                                               unique
CODIDOC                                                              
101                                                     [Matemáticas]
102                     [Informática y tecnologías de la información]
103                                                [Ciencias físicas]
104                                               [Ciencias químicas]
105                          [Ciencias de la tierra y medio ambiente]
106            [Biología (excluyendo agricultura y ciencias médicas)]
107                                        [Otras ciencias naturales]
201                                                [Ingeniería civil]
202       [Ingeniería eléctrica, electrónica y de telecomunicaciones]
203                                             [Ingeniería mecánica]
204                                              [Ingeniería química]
205                 

In [100]:
redefine('CODIDOC','titulo_doc')
df['titulo_doc_']

0            Ciencias de la salud
1               Ciencias químicas
2                      Sociología
3       Ingeniería medioambiental
4            Ciencias de la salud
                  ...            
4117        Lenguaje y literatura
4118       Historia y arqueología
4119       Historia y arqueología
4120     Ciencias de la educación
4121        Lenguaje y literatura
Name: titulo_doc_, Length: 4122, dtype: object

## Nuevas variable: 'rama_doc' 
**rama_doc**: rama de conocimiento de los estudios de doctorado. 

In [101]:
df['rama_doc']=None
df['rama_doc'].mask(df['CODIDOC'].str[0]=='6' ,1,inplace=True)
df['rama_doc'].mask(df['CODIDOC'].str[0]=='1' ,2,inplace=True)
df['rama_doc'].mask(df['CODIDOC'].str[0]=='4' ,2,inplace=True)
df['rama_doc'].mask(df['CODIDOC'].str[0]=='5' ,3,inplace=True)
df['rama_doc'].mask(df['CODIDOC'].str[0]=='2' ,4,inplace=True)
df['rama_doc'].mask(df['CODIDOC'].str[0]=='3' ,5,inplace=True)
df[['rama_doc','CODIDOC']].groupby('rama_doc').agg(['unique']) # Comprobación

,CODIDOC
,unique
rama_doc,
1,"[601, 604, 602, 603, 605]"
2,"[104, 107, 106, 105, 103, 102, 405, 101, 404, 403, 401, 402]"
3,"[504, 505, 503, 502, 507, 501, 509, 508, 506]"
4,"[207, 211, 204, 203, 205, 201, 202, 208, 209, 206, 210]"
5,"[303, 302, 305, 304, 301]"


In [102]:
# Se aplica el diccionario de ramas de conocimiento definido en la encuesta de graduados 2019
D_dict['rama_doc']=D_dict['rama_g1']
traduce('rama_doc')

                              aux_traduce
                                   unique
rama_doc                                 
1                   [Artes y humanidades]
2                              [Ciencias]
3         [Ciencias sociales y jurídicas]
4             [Ingeniería y arquitectura]
5                  [Ciencias de la salud]


## Nuevas variables: 'n_art_hum', 'n_ciencias', 'n_cc_soc_ju', 'n_ing_arq', 'n_cc_salud'
**'n_art_hum'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de artes y humanidades    
**'n_ciencias'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de Ciencias  
**'n_cc_soc_ju'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ciencias sociales y jurídicas  
**'n_ing_arq'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ingeniería y arquitectura  
**'n_cc_salud'**: Número de títulos universitarios (grado, máster, doctorado o equivalente) en la rama de ciencias de la salud  

In [103]:
L_ramas=['n_art_hum','n_ciencias','n_cc_soc_ju','n_ing_arq','n_cc_salud']
for i,rama in enumerate (L_ramas): 
    df[rama]=0
    df[rama].mask(df['rama_doc']==i+1,1,inplace=True)
df[L_ramas+['rama_doc_']].groupby('rama_doc_').agg(['unique']) # Comprobación: 

,n_art_hum,n_ciencias,n_cc_soc_ju,n_ing_arq,n_cc_salud
,unique,unique,unique,unique,unique
rama_doc_,,,,,
Artes y humanidades,[1],[0],[0],[0],[0]
Ciencias,[0],[1],[0],[0],[0]
Ciencias de la salud,[0],[0],[0],[0],[1]
Ciencias sociales y jurídicas,[0],[0],[1],[0],[0]
Ingeniería y arquitectura,[0],[0],[0],[1],[0]


## 'INFUN'
Carecter de **investigación básica o fundamental** de la investigación realizada durante el doctorado: 0=No, 1=Sí

In [104]:
df['INFUN']=df['INFUN'].astype('int') # Sin no informados
df['INFUN'].unique()

array([1, 0])

## 'INAPL'
Carecter de **investigación aplicada** de la investigación realizada durante el doctorado: 0=No, 1=Sí

In [105]:
df['INAPL']=df['INAPL'].astype('int') # Sin no informados
df['INAPL'].unique()

array([1, 0])

## 'DESTEC'
Carecter de **desarrollo experimental** de la investigación realizada durante el doctorado: 0=No, 1=Sí

In [106]:
df['DESTEC']=df['DESTEC'].astype('int') # Sin no informados
df['DESTEC'].unique()

array([1, 0])

## 'ANODOC'
**Año en que obtuvo el título de doctor**.

In [107]:
df['ANODOC']=df['ANODOC'].astype('int') # Sin no informados
df['ANODOC'].sort_values().unique()

array([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009])

## 'MESDOC'
**Mes en que obtuvo el título de doctor**.

In [108]:
df['MESDOC']=df['MESDOC'].astype('int') # Sin no informados
df['MESDOC'].sort_values().unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

## 'DURDOCANO'
**Tiempo** transcurrido desde que inició los cursos de doctorado hasta que obtuvo el **título de doctor: años completos**. 

In [109]:
df['DURDOCANO']=df['DURDOCANO'].astype('int') # Sin no informados
df['DURDOCANO'].sort_values().unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 30, 35])

## 'DURDOCMES'
**Tiempo** transcurrido desde que inició los cursos de doctorado hasta que obtuvo el **título de doctor: meses adicionales a 'DURDOCANO'**.

In [110]:
df['DURDOCMES']=df['DURDOCMES'].astype('int') # Sin no informados
df['DURDOCMES'].sort_values().unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])

In [111]:
df[df['DURDOCMES']==12] 
# Observaciones atípicas: refieren 12 meses en lugar de un año

,MUIDENT,CCAARESI,ANONAC,CCAANAC,CONTNACIM,RELA,CONTNAC1,CONTNAC2,CONTNAC3,SEXO,ESTADOCIVIL,DEPEN5,DEPEN18,DEPENMAS,NIVESTPA,NIVESTMA,NIVPROFPA,NIVPROFMA,ACTIPRIPA1,ACTIPRIMA1,EDUPRIM,EDUSECUN,BACHILLER,RES,CARES1,MESRES1,CARES2,MESRES2,CARES3,MESRES3,DOCTOR,CODIDOC,INFUN,INAPL,DESTEC,ANODOC,MESDOC,DURDOCANO,DURDOCMES,CONTEST,ANOEST,FINADOC,TRABDOC,ANOSINTRA,MESSINTRA,SITLAB,HTRAB,INGRESOS,AUTON,CNAE2,ANOINIALP,CAEMP,CONTEMP,SECTOR,CODALP,CATEGPROF,HLECT,TIPOCONT,JORLAB,BUSCATRAB,NIVELMIN,NIVELEST,RELTRABDOC,NIVELSAT1,NIVELSAT2,NIVELSAT3,NIVELSAT4,NIVELSAT5,NIVELSAT6,NIVELSAT7,NIVELSAT8,NIVELSAT9,NIVELSAT10,NIVELSAT11,NIVELSAT12,NIVELSAT13,ACTIANT,SECTORANT,ACTIPRIANT,ACTIANTANO,ACTIANTMES,TRABPOS,MOTIVOPOS1,MOTIVOPOS2,MOTIVOPOS3,MOTIVOPOS4,MOTIVOPOS5,MOTIVOPOS6,MOTIVOPOS7,MOTIVOPOS8,PORCINVES,PORCDOC,PORCOTRAS,FINANPOS,GESBUSEMP,PASOSEMP1,PASOSEMP2,PASOSEMP3,PASOSEMP4,PASOSEMP5,PASOSEMP6,PASOSEMP7,PASOSEMP8,PASOSEMP9,PASOSEMP10,PASOSEMP11,NOPASOSEMP1,NOPASOSEMP2,NOPASOSEMP3,NOPASOSEMP4,NOPASOSEMP5,NOPASOSEMP6,NOPASOSEMP7,NOPASOSEMP8,NOPASOSEMP9,CONT1,ANOIPAIS1,ANOFPAIS1,MESES1,CONT2,ANOIPAIS2,ANOFPAIS2,MESES2,CONT3,ANOIPAIS3,ANOFPAIS3,MESES3,CONT4,ANOIPAIS4,ANOFPAIS4,MESES4,CONT5,ANOIPAIS5,ANOFPAIS5,MESES5,CONT6,ANOIPAIS6,ANOFPAIS6,MESES6,CONT7,ANOIPAIS7,ANOFPAIS7,MESES7,MOTFUERA1,MOTFUERA2,MOTFUERA3,MOTFUERA4,MOTFUERA5,MOTFUERA6,MOTFUERA7,MOTVENIR1,MOTVENIR2,MOTVENIR3,MOTVENIR4,MOTVENIR5,MOTVENIR6,MOTVENIR7,PREVIRSE,MESANTESIR,MOTIRSE1,MOTIRSE2,MOTIRSE3,MOTIRSE4,MOTIRSE5,MOTIRSE6,CONTOUT,INV,MOTNOINV1,MOTNOINV2,MOTNOINV3,MOTNOINV4,MOTNOINV5,MOTNOINV6,MOTNOINV7,MOTNOINV8,MOTNOINV9,ALGINV,MOTINV1,MOTINV2,MOTINV3,MOTINV4,MOTINV5,MOTINV6,MOTINV7,MOTINV8,MOTINV9,MESESINV,ARTIC,LIBROS,NUMPAT,PATNAC,PATEU,PATCOOP,PATCOM,COMPA,TUTOR,COOPERA,INVFUT,FACTOR,año,est_encuesta,edad_años,edad,edad_,eu27,eu27_,nacionalidad,nacionalidad_,sexo,sexo_,e_civil,e_civil_,estudios_pa,estudios_pa_,grad_med_pa,grad_med_pa_,univ_pa,univ_pa_,estudios_ma,estudios_ma_,grad_med_ma,grad_med_ma_,univ_ma,univ_ma_,bach_aux,tipo_centro,tipo_centro_,doctor,doctor_,CODIDOC_,titulo_doc,titulo_doc_,rama_doc,rama_doc_,n_art_hum,n_ciencias,n_cc_soc_ju,n_ing_arq,n_cc_salud
307,70096,4,1964,04,EU27,1,EU27,None,None,1,1,0,2,0,3.0,3.0,1,1,9,9.0,1,1,1,1,04,12.0,None,0,None,0,1,101,1,1,0,1994,7,7,12,EU27,1987,5,1,1,10,1,40,5,0,85,1989,04,EU27,3,231,2,3,1,1,,2,2,3,3,3,1,1,1,3,2,1,1,2,3,1,3,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,1,,,,,,,,,200,5,0,0,0,0,0,0,0,0,0,1,11.51173913,2009,doctorado,45,3,De 35 y más años,1,Nacido en EU27,1,Nacionalidad española,0,Hombre,2,Casado,3.0,Educación primaria completa,0,Sin estudios de enseñanzas medias o superiores,0,Sin estudios universitarios,3.0,Educación primaria completa,0,Sin estudios de enseñanzas medias o superiores,0,Sin estudios universitarios,1,1,Público,1,Sí,Matemáticas,101,Matemáticas,2,Ciencias,0,1,0,0,0
1297,190035,8,1965,12,EU27,1,EU27,None,None,2,6,0,0,0,7.0,7.0,2,2,2,2.0,1,1,1,0,None,NaN,None,0,None,0,1,605,1,0,0,1995,6,1,12,EU27,1994,8,0,.,.,1,40,5,0,01,2007,08,EU27,1,100,,2,1,1,,3,3,3,2,3,2,1,1,2,2,2,2,1,2,2,2,1,1,100,1,3,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,0,,,,,,,,,1,0,,,,,,,,,,.,.,.,.,.,.,.,0,0,0,0,0,15.26447936,2009,doctorado,44,3,De 35 y más años,1,Nacido en EU27,1,Nacionalidad española,1,Mujer,1,Soltero,8.0,"Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes",1,Con estudios de enseñanzas medias o superiores,1,Con estudios universitarios,8.0,"Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes",1,Con estudios de enseñanzas medias o superiores,1,Con estudios universitarios,1,1,Público,1,Sí,Otras humanidades,605,Otras humanidades,1,Artes y humanidades,1,0,0,0,0
1485,250034,9,1963,09,EU27,1,EU27,None,None,1,1,0,2,0,2.0,8.0,1,2,4,2.0,2,1

In [112]:
# New variable (not in the original dataset):
df['Tmpo_doc']=df['DURDOCANO']+df['DURDOCMES']/12
df['Tmpo_doc'].sort_values().unique()

array([ 0.        ,  0.5       ,  1.        ,  1.08333333,  1.5       ,
        1.75      ,  2.        ,  2.08333333,  2.16666667,  2.25      ,
        2.33333333,  2.41666667,  2.5       ,  2.58333333,  2.66666667,
        2.75      ,  2.83333333,  2.91666667,  3.        ,  3.08333333,
        3.16666667,  3.25      ,  3.33333333,  3.41666667,  3.5       ,
        3.58333333,  3.66666667,  3.75      ,  3.83333333,  3.91666667,
        4.        ,  4.08333333,  4.16666667,  4.25      ,  4.33333333,
        4.41666667,  4.5       ,  4.58333333,  4.66666667,  4.75      ,
        4.83333333,  4.91666667,  5.        ,  5.08333333,  5.16666667,
        5.25      ,  5.33333333,  5.41666667,  5.5       ,  5.58333333,
        5.66666667,  5.75      ,  5.83333333,  5.91666667,  6.        ,
        6.08333333,  6.16666667,  6.25      ,  6.33333333,  6.41666667,
        6.5       ,  6.58333333,  6.66666667,  6.75      ,  6.83333333,
        6.91666667,  7.        ,  7.08333333,  7.16666667,  7.25

In [113]:
df[df['Tmpo_doc']==0] 
# Observaciones peculiares: refieren tiempo de doctorado=0. 

,MUIDENT,CCAARESI,ANONAC,CCAANAC,CONTNACIM,RELA,CONTNAC1,CONTNAC2,CONTNAC3,SEXO,ESTADOCIVIL,DEPEN5,DEPEN18,DEPENMAS,NIVESTPA,NIVESTMA,NIVPROFPA,NIVPROFMA,ACTIPRIPA1,ACTIPRIMA1,EDUPRIM,EDUSECUN,BACHILLER,RES,CARES1,MESRES1,CARES2,MESRES2,CARES3,MESRES3,DOCTOR,CODIDOC,INFUN,INAPL,DESTEC,ANODOC,MESDOC,DURDOCANO,DURDOCMES,CONTEST,ANOEST,FINADOC,TRABDOC,ANOSINTRA,MESSINTRA,SITLAB,HTRAB,INGRESOS,AUTON,CNAE2,ANOINIALP,CAEMP,CONTEMP,SECTOR,CODALP,CATEGPROF,HLECT,TIPOCONT,JORLAB,BUSCATRAB,NIVELMIN,NIVELEST,RELTRABDOC,NIVELSAT1,NIVELSAT2,NIVELSAT3,NIVELSAT4,NIVELSAT5,NIVELSAT6,NIVELSAT7,NIVELSAT8,NIVELSAT9,NIVELSAT10,NIVELSAT11,NIVELSAT12,NIVELSAT13,ACTIANT,SECTORANT,ACTIPRIANT,ACTIANTANO,ACTIANTMES,TRABPOS,MOTIVOPOS1,MOTIVOPOS2,MOTIVOPOS3,MOTIVOPOS4,MOTIVOPOS5,MOTIVOPOS6,MOTIVOPOS7,MOTIVOPOS8,PORCINVES,PORCDOC,PORCOTRAS,FINANPOS,GESBUSEMP,PASOSEMP1,PASOSEMP2,PASOSEMP3,PASOSEMP4,PASOSEMP5,PASOSEMP6,PASOSEMP7,PASOSEMP8,PASOSEMP9,PASOSEMP10,PASOSEMP11,NOPASOSEMP1,NOPASOSEMP2,NOPASOSEMP3,NOPASOSEMP4,NOPASOSEMP5,NOPASOSEMP6,NOPASOSEMP7,NOPASOSEMP8,NOPASOSEMP9,CONT1,ANOIPAIS1,ANOFPAIS1,MESES1,CONT2,ANOIPAIS2,ANOFPAIS2,MESES2,CONT3,ANOIPAIS3,ANOFPAIS3,MESES3,CONT4,ANOIPAIS4,ANOFPAIS4,MESES4,CONT5,ANOIPAIS5,ANOFPAIS5,MESES5,CONT6,ANOIPAIS6,ANOFPAIS6,MESES6,CONT7,ANOIPAIS7,ANOFPAIS7,MESES7,MOTFUERA1,MOTFUERA2,MOTFUERA3,MOTFUERA4,MOTFUERA5,MOTFUERA6,MOTFUERA7,MOTVENIR1,MOTVENIR2,MOTVENIR3,MOTVENIR4,MOTVENIR5,MOTVENIR6,MOTVENIR7,PREVIRSE,MESANTESIR,MOTIRSE1,MOTIRSE2,MOTIRSE3,MOTIRSE4,MOTIRSE5,MOTIRSE6,CONTOUT,INV,MOTNOINV1,MOTNOINV2,MOTNOINV3,MOTNOINV4,MOTNOINV5,MOTNOINV6,MOTNOINV7,MOTNOINV8,MOTNOINV9,ALGINV,MOTINV1,MOTINV2,MOTINV3,MOTINV4,MOTINV5,MOTINV6,MOTINV7,MOTINV8,MOTINV9,MESESINV,ARTIC,LIBROS,NUMPAT,PATNAC,PATEU,PATCOOP,PATCOM,COMPA,TUTOR,COOPERA,INVFUT,FACTOR,año,est_encuesta,edad_años,edad,edad_,eu27,eu27_,nacionalidad,nacionalidad_,sexo,sexo_,e_civil,e_civil_,estudios_pa,estudios_pa_,grad_med_pa,grad_med_pa_,univ_pa,univ_pa_,estudios_ma,estudios_ma_,grad_med_ma,grad_med_ma_,univ_ma,univ_ma_,bach_aux,tipo_centro,tipo_centro_,doctor,doctor_,CODIDOC_,titulo_doc,titulo_doc_,rama_doc,rama_doc_,n_art_hum,n_ciencias,n_cc_soc_ju,n_ing_arq,n_cc_salud,Tmpo_doc
469,80195,9,1951,09,EU27,1,EU27,None,None,2,1,0,2,0,2.0,2.0,1,5,7,NaN,2,2,3,1,09,12.0,None,0,None,0,1,601,0,1,1,1991,1,0,0,EU27,1973,5,1,0,0,1,40,7,0,91,1992,09,EU27,2,262,,1,1,1,,3,3,1,2,2,1,1,1,1,1,2,2,2,3,2,3,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,,,,,,,,1,,20,0,0,0,0,0,0,0,0,0,1,1,25.66619877,2009,doctorado,58,3,De 35 y más años,1,Nacido en EU27,1,Nacionalidad española,1,Mujer,2,Casado,2.0,Educación primaria incompleta (ha asistido menos de 5 años a la escuela),0,Sin estudios de enseñanzas medias o superiores,0,Sin estudios universitarios,2.0,Educación primaria incompleta (ha asistido menos de 5 años a la escuela),0,Sin estudios de enseñanzas medias o superiores,0,Sin estudios universitarios,3,0,Privado,1,Sí,Historia y arqueología,601,Historia y arqueología,1,Artes y humanidades,1,0,0,0,0,0.0
2120,280690,13,1942,07,EU27,1,EU27,None,None,1,1,0,0,0,9.0,3.0,1,5,3,NaN,3,3,3,1,13,12.0,None,0,None,0,1,201,1,1,0,1992,10,0,0,EU27,1966,9,1,0,0,1,35,8,0,85,1997,13,EU27,3,231,2,3,1,1,,2,1,1,2,2,1,1,1,2,1,1,1,1,2,1,1,0,,NNN,.,.,0,,,,,,,,,.,.,.,,,,,,,,,,,,,,,,,,,,,,,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,NMOV,.,.,.,,,,,,,,,,,,,,,3,,,,,,,,NMOV,1,,,,,,,,,,,1,,,,,1,,1,,432,0,9,0,0,0,0,0,0,0,0,1,24.81791986,2009,doctorado,67,3,De 35 y más años,1,Nacido en EU27,1,Nacionalidad española,0,Hombre,2,Casado,8.0,"Estudios universitarios (Diplomatura, Licenciatura, Doctorado) o equivalentes",1,Con estudios de enseñanzas medias o superiores,1,Con estudios universitarios,3.0,Educación primaria completa,0,Sin estudios de enseñanzas medias o superiores,0,Sin estudios universitarios,3,0,Privado,1,Sí,Ingeniería civil,201,Ingeniería civil,4,Ingeniería y arquitectura

## 'CONTEST'
**Continente** en el que realizó los **estudios universitarios de segundo ciclo** previos al doctorado: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NNNN'=Blanco.

In [114]:
df['CONTEST'].sort_values().unique() # Sin no informados

array(['AFRI', 'ASIA', 'EU27', 'EUCA', 'RAME', 'REUR'], dtype=object)

## 'ANOEST'
**Año en el que finalizó los estudios universitarios de segundo ciclo**.

In [115]:
df['ANOEST']=df['ANOEST'].astype('int') # Sin no informados
df['ANOEST'].sort_values().unique()

array([1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975,
       1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986,
       1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997,
       1998, 1999, 2000, 2001, 2002, 2003, 2004])

## 'FINADOC'
**Fuente principal de financiación** durante los **estudios de doctorado**: 1=Beca de la Administración Pública (central, autonómica, local,etc.), 2=Beca de otro tipo de institución española, 3=Beca de una Administración Pública extranjera, 4=Beca de otro tipo de institución extranjera, 5=Trabajó como profersor y/o ayudante de investigación, 6=Otra ocupación, 7=Subvención reembolsada por el empleador, 8=Préstamos, ahorros personales y/o ayuda familiar, 9=Otras formas. 

In [116]:
df['FINADOC']=df['FINADOC'].astype('int') # Sin no informados
df['FINADOC'].sort_values().unique()

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

## 'TRABDOC'
¿**Trabajó** en algo **relacionado con su doctorado** una vez finalizado y antes de enero de 2010?: 0=No, 1=Sí

In [117]:
df['TRABDOC']=df['TRABDOC'].astype('int') # Sin no informados
df['TRABDOC'].sort_values().unique()

array([0, 1])

## 'ANOSINTRA'
(Solo para respuesta positiva en variable 'TRABDOC')  

**Tiempo** transcurrido **desde la finalización de los estudios de doctorado hasta que empezó a trabajar en algo relacionado con estos**: **años** completos. 

In [118]:
df['ANOSINTRA'].sort_values().unique() # No informados como ' .'

array([' .', ' 0', ' 1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9',
       '10', '12', '14', '15', '16', '17', '19'], dtype=object)

In [119]:
df[['ANOSINTRA','TRABDOC']].groupby('TRABDOC').agg(['unique']) 

,ANOSINTRA
,unique
TRABDOC,
0,[ .]
1,"[ 0, 1, 2, 4, 3, 14, 5, 16, 7, 15, 6, 9, 10, 8, 19, 12, 17]"


In [120]:
df['ANOSINTRA'].replace({' .':None},inplace=True) 
df['ANOSINTRA']=df['ANOSINTRA'].astype('float')
df['ANOSINTRA'].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 12., 14.,
       15., 16., 17., 19., nan])

## 'MESSINTRA'
(Solo para respuesta positiva en variable 'TRABDOC')   

**Tiempo** transcurrido **desde la finalización de los estudios de doctorado hasta que empezó a trabajar en algo relacionado con estos**: **meses** adicionales a 'ANOSINTRA'.

In [121]:
df['MESSINTRA'].sort_values().unique() # No informados como  ' .'

array([' .', ' 0', ' 1', ' 2', ' 3', ' 4', ' 5', ' 6', ' 7', ' 8', ' 9',
       '10', '11', '12'], dtype=object)

In [122]:
df[['MESSINTRA','TRABDOC']].groupby('TRABDOC').agg(['unique']) 

,MESSINTRA
,unique
TRABDOC,
0,[ .]
1,"[ 0, 1, 9, 8, 10, 3, 6, 5, 2, 7, 4, 11, 12]"


In [123]:
df['MESSINTRA'].replace({' .':None},inplace=True) 
df['MESSINTRA']=df['MESSINTRA'].astype('float')
df['MESSINTRA'].sort_values().unique()

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       nan])

## Nueva variable: 'Tmpo_s/T'
(Solo para respuesta positiva en variable 'TRABDOC')   

**Tiempo** transcurrido **desde la finalización de los estudios de doctorado hasta que empezó a trabajar en algo relacionado con estos**: en meses.

In [124]:
df['Tmpo_s/T']=df['ANOSINTRA']+df['MESSINTRA']/12
df['Tmpo_s/T'].sort_values().unique() # valores nulos (a reconsiderar)

array([ 0.        ,  0.08333333,  0.16666667,  0.25      ,  0.33333333,
        0.41666667,  0.5       ,  0.58333333,  0.66666667,  0.75      ,
        0.83333333,  0.91666667,  1.        ,  1.08333333,  1.16666667,
        1.25      ,  1.33333333,  1.41666667,  1.5       ,  1.58333333,
        1.66666667,  1.75      ,  1.83333333,  1.91666667,  2.        ,
        2.08333333,  2.16666667,  2.25      ,  2.33333333,  2.41666667,
        2.5       ,  2.58333333,  2.66666667,  2.75      ,  2.83333333,
        2.91666667,  3.        ,  3.16666667,  3.25      ,  3.33333333,
        3.41666667,  3.5       ,  3.58333333,  3.66666667,  3.75      ,
        3.83333333,  3.91666667,  4.        ,  4.08333333,  4.16666667,
        4.25      ,  4.33333333,  4.41666667,  4.58333333,  4.83333333,
        5.        ,  5.08333333,  5.16666667,  5.25      ,  5.33333333,
        5.5       ,  5.66666667,  6.        ,  6.16666667,  6.5       ,
        6.75      ,  7.        ,  7.16666667,  7.25      ,  7.75

## 'SITLAB' (redefinida como 'sit_lab')
**Situación laboral** el 31 de diciembre de 2009: 1=Empleado, 2=Desempleado, 3=Inactivo.

In [125]:
df['SITLAB']=df['SITLAB'].astype('int') # Sin no informados
df['SITLAB'].unique()

array([1, 2, 3])

In [126]:
df['sit_lab']=df['SITLAB']

In [127]:
traduce('sit_lab')

            aux_traduce
                 unique
sit_lab                
1          [Trabajando]
2        [En desempleo]
3            [Inactivo]


## Nueva variable: 'sit_lab_detalle'
**Detalle de situación laboral** de acuerdo con las categorías definidas para los datasets de graduados y titulados de máster universitario. 

In [128]:
leyenda('sit_lab_detalle')

1 	 (Inactivo) Estudiante (estudiante o preparando oposiciones) 
2 	 (Inactivo) Jubilado
3 	 (Inactivo) Incapacitado para trabajar
9 	 (Ocupado) Asalariado con trabajo temporal o contrato de trabajo de duración determinada
4 	 (Inactivo) Dedicado a las labores del hogar
5 	 (Inactivo) Otra situación
11 	 (Ocupado) Trabajador independiente o empresario sin asalariados
6 	 Desempleado
7 	 (Ocupado) Trabajador en prácticas, formación (incluido MIR, EIR, FIR,…) o becario
8 	 (Ocupado) Asalariado con trabajo permanente o contrato de trabajo de duración indefinida
10 	 (Ocupado) Empresario con asalariados
12 	 (Ocupado) Ayuda en la empresa o negocio familiar


In [129]:
sdfghjklñlkjhbgvfd

NameError: name 'sdfghjklñlkjhbgvfd' is not defined

## 'HTRAB'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Número medio de **horas semanales trabajadas** en diciembre de 2009. 

In [ ]:
df['HTRAB'].sort_values().unique() # No informados como ' .'

In [ ]:
df[['SITLAB','HTRAB']].groupby('SITLAB').agg(['unique']) 

In [ ]:
df['HTRAB'].replace({' .':0},inplace=True) 
df['HTRAB']=df['HTRAB'].astype('int')
df['HTRAB'].sort_values().unique()

In [ ]:
# función re_sitlab: inspección e imputacion de valores de variables condicionadas por respuesta a 'SITLAB'
def re_sitlab(col,blank,value=None,tipus='float'):
    print (df[['SITLAB',col]].groupby('SITLAB').agg(['unique']))
    df[col].replace({blank:value},inplace=True) 
    df[col]=df[col].astype(tipus)
    print (df[['SITLAB',col]].groupby('SITLAB').agg(['unique']))
    return 

## 'INGRESOS'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB') 

Tramo en que se encontraban sus **ingresos brutos anuales** en 2009: 1=Menos de 10.000 euros, 2=De 10.000 a 20.000 euros, 3=De 20.001 a 30.000 euros, 4=De 30.001 a 35.000 euros, 5=De 35.001 a 40.000 euros, 6=De 40.001 a 45.000 euros, 7=De 45.001 a 50.000 euros, 8=Más de 50.000 euros. 

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@  
Pendiente: convertir en sueldo mensual NETO

In [ ]:
df['INGRESOS'].sort_values().unique() # Blank as ' '

In [ ]:
re_sitlab('INGRESOS',blank=' ',value=0,tipus='int')

In [ ]:
df['INGRESOS'].dtypes

## 'AUTON'  (informa la variable 'sit_lab_detalle')
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

¿El 31 de diciembre de 2009 su actividad laboral principal era **trabajador por cuenta propia**? 0=No, 1=Sí. 

In [ ]:
df['AUTON'].sort_values().unique() # No informados como ' '

In [ ]:
re_sitlab('AUTON',blank=' ',value=0,tipus='int')

## 'CNAE2'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Descripción de la **actividad principal** de la empresa en la que trabaja de acuerdo con la clasificación CNAE-2009. 

In [ ]:
df['CNAE2'].sort_values().unique() # No informados como 'NN'

In [ ]:
re_sitlab('CNAE2',blank='NN')

## 'ANOINIALP'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB') 

**Año en que empezó a trabajar** en la organización.  

In [ ]:
df['ANOINIALP'].sort_values().unique() # No informados como '   .'

In [ ]:
re_sitlab('ANOINIALP',blank='   .')

## 'CAEMPP'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

**CCAA** en la que desarrolla su **actividad laboral principal** el 31 de diciembre de 2009: '01'=Andalucía, '02'=Aragón, '03'=Asturias (Principado de), '04'=Balears (Illes), '05'=Canarias, '06'=Cantabria, '07'=Castilla y León, '08'=Castilla - La Mancha, '09'=Cataluña, '10'=Comunidad Valenciana, '11'=Extremadura, '12'=Galicia, '13'=Madrid (Comunidad de), '14'=Murcia (Región de), '15'=Navarra (Comunidad Foral de), '16'=País Vasco, '17'=Rioja (La), '18'=Ceuta y Melilla. 

In [ ]:
df['CAEMP'].sort_values().unique() # No informados como 'NN'

In [ ]:
re_sitlab('CAEMP',blank='NN')

## 'CONTEMP'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

**Continente** en la que desarrolla su **actividad laboral principal** el 31 de diciembre de 2009: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NNNN'=Blanco.

In [ ]:
df['CONTEMP'].sort_values().unique() # No informados como 'NNNN'

In [ ]:
re_sitlab('CONTEMP',blank='NNNN',tipus='object')

## 'SECTOR'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

**Sector** al que pertenece la organización en la que trabajaba el 31 de diciembre de 2009: 1=Empresas, 2=AAPP, 3=Enseñanza superior, 4=IPSFL.

In [ ]:
df['SECTOR'].sort_values().unique() # No informados como ' '

In [ ]:
re_sitlab('SECTOR',blank=' ')

## 'CODALP'
(Solo para respuesta 'Empleado' en variable 'SITLAB')  

**Actividad laboral** que desarrollaba el 31 de diciembre de 2009 de acuerdo con la Clasificación internacional uniforme de ocupaciones (ISCO-08): 

100=Directores y gerentes

Profesionales científicos e intelectuales: 211=Físicos, químicos y profesionales afines, 212=Matemáticos, actuarios y estadísticos, 213=Profesionales en ciencias biológicas, 214=Ingenieros ( excluyendo electrotecnólogos), 215=Ingenieros en electrotecnología, 216=Arquitectos, urbanistas, agrimensores y diseñadores, 221=Médicos, 222=Profesionales de enfermería y partería, 223=Profesionales de medicina tradicional y alternativa, 224=Practicantes paramédicos, 225=Practicantes, paramédicos, 226=Otros profesionales de la salud, 231=Profesores de universidades y otros establecimientos de la enseñanza superior, 232=Profesores de formación profesional, 233=Profesores de enseñanza secundaria, 234=Maestros de enseñanza primaria y maestros preescolares, 235=Otros profesionales de la enseñanza, 241=Especialistas en finanzas, 242=Especialistas en organización de administración, 243=Profesionales de las ventas, la comercialización y las relaciones públicas, 251=Desarrolladores y analistas de software y multimedia, 252=Especialistas en bases de datos y en redes de computadores, 261=Profesionales en derecho, 262=Archivistas, bibliotecarios, curadores y afines, 263=Especialistas en ciencias sociales y teología
264=Autores, periodistas y lingüistas, 265=Artistas creativos e interpretativos

Técnicos y profesionales de nivel medio: 311=Técnicos en ciencias físicas y en ingeniería, 312=Supervisores en ingeniería de minas, de industrias manufactureras y de la construcción, 313=Técnicos en control de procesos, 314=Técnicos y profesionales de nivel medio en ciencias biológicas y afines, 315=Técnicos y controladores en navegación marítima y aeronáutica, 400=Personal de apoyo administrativo, 500=Trabajadores de los servicios y vendedores de comercios y mercados, 600=Agricultores y trabajadores calificados agropecuarios y pesqueros
700=Oficiales, operarios y artesanos de artes mecánicas y de otros oficios, 800=Operadores de instalaciones y máquinas y ensambladores, 900=Ocupaciones elementales, 000=Ocupaciones militares

In [ ]:
df['CODALP'].sort_values().unique() # No informados como 'NNN'

In [ ]:
df['CODALP'].nunique()

In [ ]:
re_sitlab('CODALP',blank='NNN')

## 'CATEGPROF'
(Solo para respuesta 231 en variable 'CODALP')  

**Categoría profesional o puesto de trabajo** que ocupaba el 31 de diciembre de 2009: 1=Catedrático, 2=Profesor titular, 3=Profesor asociado, emerito, visitante, ayudante o similar, 4=Otro.

In [ ]:
df['CATEGPROF'].sort_values().unique() # No informados como ' '

In [ ]:
df[['CODALP','CATEGPROF']].groupby('CODALP',dropna=False).agg(['unique']) # Inspección

In [ ]:
df['CATEGPROF'].replace({' ':np.nan},inplace=True) # None para 'CODALP' distinto de 231
df['CATEGPROF']=df['CATEGPROF'].astype('float')
df['CATEGPROF'].sort_values().unique()

## 'HLECT'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Proporción de su trabajo dedicado a **actividades de docencia**: 1=Nada, 2=Menos del 25 %; 3=Del 25 a 50 %; 4=Del 50 al 75 %; 5=Más del 75%. 

In [ ]:
df['HLECT']=df['HLECT']
df['HLECT'].sort_values().unique() # No informados como ' '

In [ ]:
re_sitlab('HLECT',blank=' ')

## 'TIPOCONT' (informa la variable 'sit_lab_detalle')
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

**Tipo de contrato** al que estaba sujeto: 1=Indefinido, 2=Temporal. 

In [ ]:
leyenda('sit_lab_detalle')

In [ ]:
df['TIPOCONT'].sort_values().unique() # no informados como ' '

In [ ]:
df[['SITLAB','TIPOCONT','AUTON']].groupby(['SITLAB','AUTON']).agg(['unique']) # Inspección

In [ ]:
df['TIPOCONT'].replace({' ':None},inplace=True) 
df['TIPOCONT']=df['TIPOCONT'].astype('float')
df['TIPOCONT'].sort_values().unique()

#### 'JORLAB'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB') 

Tipo de **jornada laboral**: 1=A tiempo completo, 2=A tiempo parcial.

In [ ]:
df['JORLAB'].sort_values().unique() # Blank as ' '

In [ ]:
df[['SITLAB','JORLAB']].groupby('SITLAB').agg(['unique']) # No missing values

In [ ]:
df['JORLAB'].replace({' ':None},inplace=True) # None for 'SITLAB' different from 1 
df['JORLAB']=df['JORLAB'].astype('float')
df['JORLAB'].sort_values().unique()

#### 'BUSCATRAB'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y respuesta 'Tiempo parcial' (2) en variable 'JORLAB') 

¿Estaba usted **buscando un trabajo a tiempo completo** en diciembre de 2009? 0=No, 1=Sí.

In [ ]:
df['BUSCATRAB'].sort_values().unique() # Blank as ' '

In [ ]:
df[['SITLAB','JORLAB','BUSCATRAB']].groupby(['SITLAB','JORLAB'],dropna=False).agg(['unique']) 
#  No missing values

In [ ]:
df['BUSCATRAB'].replace({' ':None},inplace=True) 
# None for 'SITLAB' different from 1 and 'JORLAB' different from 2
df['BUSCATRAB']=df['BUSCATRAB'].astype('float')
df['BUSCATRAB'].sort_values().unique()

#### 'NIVELMIN'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB') 

**Nivel mínimo requerido para el puesto** que tenía usted en su trabajo principal el 31 de diciembre de 2009: 1=Postdoctorado, 2=Doctor universitario, 3=Licenciado, arquitecto, ingeniero, 4=Diplomado, arquitecto técnico, ingeniero técnico, 5=Ciclos formativos de grado superior, 6=Ciclos formativos de grado medio,bachiller, 7= Otros.

In [ ]:
df['NIVELMIN'].sort_values().unique() # Blank as ' '

In [ ]:
re_sitlab('NIVELMIN',blank=' ')

#### 'NIVELEST'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

**Nivel** que considera que es el **adecuado para el puesto** que tenía usted en su trabajo principal el 31 de
diciembre de 2009: 1=Postdoctorado, 2=Doctor universitario, 3=Licenciado, arquitecto, ingeniero, 4=Diplomado, arquitecto técnico, ingeniero técnico, 5=Ciclos formativos de grado superior, 6=Ciclos formativos de grado medio,bachiller, 7= Otros.

In [ ]:
df['NIVELEST'].sort_values().unique() # Blank as ' '

In [ ]:
re_sitlab('NIVELEST',blank=' ')

#### 'RELTRABDOC'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Grado de **relación entre el trabajo principal** que desempeñaba a 31 de diciembre de 2009 **y sus estudios
de doctorado**:1=Alto, 2=Medio, 3=Bajo.

In [ ]:
df['RELTRABDOC'].sort_values().unique() # Blank as ' '

In [ ]:
re_sitlab('RELTRABDOC',blank=' ')

#### 'NIVELSAT1'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Nivel de **satisfacción con su salario** el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [ ]:
df['NIVELSAT1'].sort_values().unique() # Blank as ' '

In [ ]:
re_sitlab('NIVELSAT1',blank=' ')

#### 'NIVELSAT2'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Nivel de **satisfacción con sus beneficios económicos** el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [ ]:
re_sitlab('NIVELSAT2',blank=' ')

#### 'NIVELSAT3'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')

Nivel de **satisfacción con su estabilidad laboral** el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [ ]:
re_sitlab('NIVELSAT3',blank=' ')

#### 'NIVELSAT4'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Nivel de **satisfacción con su localización laboral** el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [ ]:
re_sitlab('NIVELSAT4',blank=' ')

#### 'NIVELSAT5'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Nivel de **satisfacción con sus condiciones laborales** el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [ ]:
re_sitlab('NIVELSAT5',blank=' ')

#### 'NIVELSAT6'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Nivel de **satisfacción con sus oportunidades para promocionar** el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [ ]:
re_sitlab('NIVELSAT6',blank=' ')

#### 'NIVELSAT7'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Nivel de **satisfacción con el componente o reto intelectual** el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [ ]:
re_sitlab('NIVELSAT7',blank=' ')

#### 'NIVELSAT8'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Nivel de **satisfacción con su nivel de responsabilidad** el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [ ]:
re_sitlab('NIVELSAT8',blank=' ')

#### 'NIVELSAT9'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Nivel de **satisfacción con su grado de independencia** el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [ ]:
re_sitlab('NIVELSAT9',blank=' ')

#### 'NIVELSAT10'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  
Nivel de **satisfacción con su contribución a la sociedad** el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [ ]:
re_sitlab('NIVELSAT10',blank=' ')

#### 'NIVELSAT11'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Nivel de **satisfacción con su estatus social** el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [ ]:
re_sitlab('NIVELSAT11',blank=' ')

#### 'NIVELSAT12'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

Nivel de **satisfacción con sus conciliación horario laboral/actividades personales** el 31 de diciembre de 2009: 1=Alto, 2=Medio, 3=Bajo, 4=Ninguno.

In [ ]:
re_sitlab('NIVELSAT12',blank=' ')

#### 'NIVELSAT13'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

**Evaluación global de satisfacción** el 31 de diciembre de 2009: 1=Alta, 2=Media, 3=Baja, 4=Ninguna.

In [ ]:
re_sitlab('NIVELSAT13',blank=' ')

#### 'ACTIANT'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

En los últimos 5 años, ¿ha desempeñado **alguna actividad laboral distinta** de la que desarrollaba a 31 de diciembre
de 2009? 0=No, 1=Sí.

In [ ]:
df['ACTIANT']=df['ACTIANT']
df['ACTIANT'].sort_values().unique() # Blank as ' '

In [ ]:
re_sitlab('ACTIANT',blank=' ')

#### 'SECTORANT'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y respuesta 'Sí' (1) en la variable 'ACTIANT')  

**Sector** al que pertenece la organización en la que trabajaba en su **actividad anterior**: 1=Empresas, 2=AAPP, 3=Enseñanza superior, 4=IPSFL.

In [ ]:
df['SECTORANT'].sort_values().unique() # Blank as ' '

In [ ]:
# Función re_actiant: inspección e imputación de valores de variables condicionadas por respuesta a 'ACTIANT'
def re_actiant(col,blank,value=None,tipus='float'):
    print (df[['SITLAB','ACTIANT',col]].groupby(['SITLAB','ACTIANT'],dropna=False).agg(['unique']))
    df[col].replace({blank:value},inplace=True) 
    df[col]=df[col].astype(tipus)
    print (df[['SITLAB','ACTIANT',col]].groupby(['SITLAB','ACTIANT'],dropna=False).agg(['unique']))
    return 

In [ ]:
re_actiant('SECTORANT',blank=' ')

#### 'ACTIPRIANT'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y respuesta 'Sí' (1) en la variable 'ACTIANT')  

**Actividad laboral** que desarrollaba en su **ocupación anterior** de acuerdo con la Clasificación internacional uniforme de ocupaciones (ISCO-08) (Códigos: véase la variable 'CODALP').

In [ ]:
df['ACTIPRIANT'].sort_values().unique() # Blank as 'NNN'

In [ ]:
re_actiant('ACTIPRIANT',blank='NNN')

#### 'ACTIANTANO'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y respuesta 'Sí' (1) en la variable 'ACTIANT')  

Duración de su **ocupación anterior: años completos**.

In [ ]:
df['ACTIANTANO'].sort_values().unique() # Blank as ' .'

In [ ]:
re_actiant('ACTIANTANO',blank=' .')

#### 'ACTIANTMES'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y respuesta 'Sí' (1) en la variable 'ACTIANT')  

Duración de su **ocupación anterior: meses** que exceden los años declarados en 'ACTIANTANO'.

In [ ]:
df['ACTIANTMES'].sort_values().unique() # Blank as ' .'

In [ ]:
re_actiant('ACTIANTMES',blank=' .')

In [ ]:
# New variable (not in original dataset):
df['Temp_act_ant']=df['ACTIANTANO']+df['ACTIANTMES']/12
df['Temp_act_ant'].sort_values().unique() # VALORES NULOS. ¿DESCARTAR?

#### 'TRABPOS'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB')  

¿Era un **puesto postdoctoral** su principal empleo el 31 de diciembre de 2009? 0=No, 1=Sí. 

In [ ]:
df['TRABPOS'].sort_values().unique() # Blank as ' '

In [ ]:
re_sitlab('TRABPOS',blank=' ')

#### 'MOTIVOPOS1'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  

¿Era **realizar prácticas en su campo de estudio** una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [ ]:
df['MOTIVOPOS1'].sort_values().unique() # Blank as ' '

In [ ]:
# Función re_trabpos: Inspección e imputación de valores de variables condicionadas por respuesta a 'TRABPOS'
def re_trabpos(col,blank=' ',value=None,tipus='float'):
    print (df[['SITLAB','TRABPOS',col]].groupby(['SITLAB','TRABPOS'],dropna=False).agg(['unique']))
    df[col].replace({blank:value},inplace=True) 
    df[col]=df[col].astype(tipus)
    print (df[['SITLAB','TRABPOS',col]].groupby(['SITLAB','TRABPOS'],dropna=False).agg(['unique']))
    return 

In [ ]:
re_trabpos('MOTIVOPOS1')

#### 'MOTIVOPOS2'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  

¿Era **realizar prácticas en un campo diferente al suyo** una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [ ]:
re_trabpos('MOTIVOPOS2')

#### 'MOTIVOPOS3'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  

¿Era **realizar investigación de forma independiente** una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [ ]:
re_trabpos('MOTIVOPOS3')

#### 'MOTIVOPOS4'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  

¿Era **trabajar con una persona específica o en un lugar concreto** una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [ ]:
re_trabpos('MOTIVOPOS4')

#### 'MOTIVOPOS5'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  

¿Era **llevar a cabo actividades docentes** una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [ ]:
re_trabpos('MOTIVOPOS5')

#### 'MOTIVOPOS6'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  

¿Era **no haber podido elegir otro empleo** una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [ ]:
re_trabpos('MOTIVOPOS6')

#### 'MOTIVOPOS7'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  

¿Era que es el **trabajo que se suele realizar en su campo de estudio** una de las razones para realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [ ]:
re_trabpos('MOTIVOPOS7')

#### 'MOTIVOPOS8'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  

¿Eran **otras razones** las que le llevaron a realizar ese trabajo de postdoctorado? 0=No, 1=Sí. 

In [ ]:
re_trabpos('MOTIVOPOS8')

#### 'PORCINVES'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  

Porcentaje de su tiempo como postdoctorado empleado en **actividades de investigación**.

In [ ]:
df['PORCINVES'].sort_values().unique() # Blank as '  .'

In [ ]:
re_trabpos('PORCINVES',blank='  .')

#### 'PORCDOC'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  

Porcentaje de su tiempo como postdoctorado empleado en **actividades de docencia**.

In [ ]:
df['PORCDOC'].sort_values().unique() # Blank as '  .'

In [ ]:
re_trabpos('PORCDOC',blank='  .')

#### 'PORCOTRAS'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  

Porcentaje de su tiempo como postdoctorado empleado en **otras actividades**.

In [ ]:
df['PORCOTRAS'].sort_values().unique() # Blank as '  .'

In [ ]:
re_trabpos('PORCOTRAS',blank='  .')

In [ ]:
# Variable temporal para comprobacion:
df['Temp_porc']=df['PORCINVES']+df['PORCDOC']+df['PORCOTRAS']
df['Temp_porc'].sort_values().unique()

In [ ]:
del df['Temp_porc']

#### 'FINANCPOS'
(Solo para respuesta 'Empleado' (1) en variable 'SITLAB' y 'Sí' (1) en la variable 'TRABPOS')  

Fuente principal de **financiación** de ese postdoctorado: 1=Empresas, 2=AAPP, 3=Centros de enseñanza superior, 4=IPSFL, 5=Fundaciones privadas, 6=OTRAS.

In [ ]:
df['FINANPOS'].sort_values().unique() # Blank as ' '

In [ ]:
re_trabpos('FINANPOS')

#### 'GESBUSEMP'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB')  

Durante enero de 2010, ¿hizo alguna **gestión para buscar empleo**? 0=No, 1=Sí.

In [ ]:
df['GESBUSEMP'].sort_values().unique() # Blank as ' '

In [ ]:
re_sitlab('GESBUSEMP',blank=' ')

#### 'PASOSEMP1'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  

¿Se puso en contacto con una **empresa de trabajo temporal**? 1=Sí, blanco.

In [ ]:
df['PASOSEMP1'].sort_values().unique() # Blank as ' '

In [ ]:
df[['SITLAB','GESBUSEMP','PASOSEMP1']].groupby(['SITLAB','GESBUSEMP'],dropna=False).agg(['unique']) 

In [ ]:
# Función pasosemp: inspeccion e imputación de valores de las variables 'PASOSEMPi' (i=1,2,...11)
def pasosemp(col):
    print (df[['SITLAB','GESBUSEMP',col]].groupby(['SITLAB','GESBUSEMP'],dropna=False).agg(['unique']))
    # Se imputa respuesta 'No' (0) a quien no contesta sí habiendo sido preguntado:
    df[col].mask((df['GESBUSEMP']==1)&(df[col]==' '),0,inplace=True) 
    # Se imputa None cuando no se formula la pregunta:
    df[col].where((df[col]==0)|(df[col]=='1'),None,inplace=True)
    df[col]=df[col].astype('float')
    print (df[['SITLAB','GESBUSEMP',col]].groupby(['SITLAB','GESBUSEMP'],dropna=False).agg(['unique']))
    return 

In [ ]:
pasosemp('PASOSEMP1')

#### 'PASOSEMP2'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  

¿Buscó en **oficinas de empleo**? 1=Sí, blanco.

In [ ]:
pasosemp('PASOSEMP2')

#### 'PASOSEMP3'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  

¿Se puso en contacto directamente con las **empresas** que necesitan personal? 1=Sí, blanco.

In [ ]:
pasosemp('PASOSEMP3')

#### 'PASOSEMP4'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  

¿Revisó y respondió **anuncios** de periódicos? 1=Sí, blanco.

In [ ]:
pasosemp('PASOSEMP4')

#### 'PASOSEMP5'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  

¿Buscó en **Internet**? 1=Sí, blanco.

In [ ]:
pasosemp('PASOSEMP5')

#### 'PASOSEMP6'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  

¿Buscó ayuda entre **amigos y parientes**? 1=Sí, blanco.

In [ ]:
pasosemp('PASOSEMP6')

#### 'PASOSEMP7'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  

¿Buscó ayuda entre **profesionales conocidos del sector**? 1=Sí, blanco.

In [ ]:
pasosemp('PASOSEMP7')

#### 'PASOSEMP8'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  

¿Buscó terrenos, maquinaria o equipamiento para establecer **su propia empresa**? 1=Sí, blanco.

In [ ]:
pasosemp('PASOSEMP8')

#### 'PASOSEMP9'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  

¿Hizo **entrevistas de trabajo**? 1=Sí, blanco.

In [ ]:
pasosemp('PASOSEMP9')

#### 'PASOSEMP10'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  

¿Solicitó **permisos, licencias o recursos financieros**? 1=Sí, blanco.

In [ ]:
pasosemp('PASOSEMP10')

#### 'PASOSEMP11'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'Sí' (1) en la variable 'GESBUSEMP')  

¿Realizó **otras** gestiones? 1=Sí, blanco.

In [ ]:
pasosemp('PASOSEMP11')

#### 'NOPASOSEMP1'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  

¿La **enfermedad o discapacidad** fueron una razón para no buscar empleo? 1=Sí, blanco.

In [ ]:
df['NOPASOSEMP1'].sort_values().unique() # Blank as ' '

In [ ]:
df[['SITLAB','GESBUSEMP','NOPASOSEMP1']].groupby(['SITLAB','GESBUSEMP'],dropna=False).agg(['unique']) 

In [ ]:
# Función mopasosemp: inspeccion e imputación de valores de las variables 'NOPASOSEMPi' (i=1,2,...9)
def nopasosemp(col):
    print (df[['SITLAB','GESBUSEMP',col]].groupby(['SITLAB','GESBUSEMP'],dropna=False).agg(['unique']))
    # Se imputa respuesta 'No' (0) a quien no contesta sí habiendo sido preguntado:
    df[col].mask((df['GESBUSEMP']==0)&(df[col]==' '),0,inplace=True) 
    # Se imputa None cuando no se formula la pregunta:
    df[col].where((df[col]==0)|(df[col]=='1'),None,inplace=True)
    df[col]=df[col].astype('float')
    print (df[['SITLAB','GESBUSEMP',col]].groupby(['SITLAB','GESBUSEMP'],dropna=False).agg(['unique']))
    return 

In [ ]:
nopasosemp('NOPASOSEMP1')

#### 'NOPASOSEMP2'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  

¿El **cuidado de hijos** fue una razón para no buscar empleo? 1=Sí, blanco.

In [ ]:
nopasosemp('NOPASOSEMP2')

#### 'NOPASOSEMP3'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  

¿El **cuidado de otras personas dependientes** fue una razón para no buscar empleo? 1=Sí, blanco.

In [ ]:
nopasosemp('NOPASOSEMP3')

#### 'NOPASOSEMP4'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  

¿Fueron **otras responsabilidades familiares** una razón para no buscar empleo? 1=Sí, blanco.

In [ ]:
nopasosemp('NOPASOSEMP4')

#### 'NOPASOSEMP5'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  

¿Fue la **jubilación** una razón para no buscar empleo? 1=Sí, blanco.

In [ ]:
nopasosemp('NOPASOSEMP5')

#### 'NOPASOSEMP6'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  

¿Fue la **asistencia a instituciones educativas** una razón para no buscar empleo? 1=Sí, blanco.

In [ ]:
nopasosemp('NOPASOSEMP6')

#### 'NOPASOSEMP7'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  

¿Fue la **convicción** de que no existe ningún trabajo disponible para usted una razón para no buscar empleo? 1=Sí, blanco.

In [ ]:
nopasosemp('NOPASOSEMP7')

#### 'NOPASOSEMP8'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  

¿Fue que **no necesita o no quiere trabajar** una razón para no buscar empleo? 1=Sí, blanco.

In [ ]:
nopasosemp('NOPASOSEMP8')

#### 'NOPASOSEMP9'
(Solo para respuestas 'Desmpleado' (2) e 'Inactivo' (3) en variable 'SITLAB' y 'No' (0) en la variable 'GESBUSEMP')  

¿Tenía **otros** motivos para no buscar empleo? 1=Sí, blanco.

In [ ]:
nopasosemp('NOPASOSEMP9')

#### 'CONT1'

**Continente del primer país de residencia** -en orden cronológico inverso- durante **al menos 3 meses en el periodo 2000-2009**: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NMOV'=Sin movilidad.

**Observaciones**: 
'CONT1' es la primera de 7 variables 'CONTi' (i=1,2,...,7). Sus valores se construyen a partir de las respuestas a la petición de enumerar los países de residencia en el periodo 2000-2009. Solo se pide esta relación de países a quienes han vivido algún tiempo fuera de España en el periodo: 
- El INE ha imputado 'NMOV' en todas las columnas 'CONTi' (i=1,2,...,7) a quienes no aportan una relación de países por no haber vivido fuera de España en el periodo de referencia. 
- 'NMOV' también es el valor con el que se han relleado los continentes 'CONTi'  cuando se agotan los países referidos por el encuestado.
- 'La respuesta 'España' en la relación de países de residencia parece haber sido sustituida por 'EU27' o, al menos en una ocasión, por 'NMOV' (véanse las observaciones atípicas de la columna 'CONT2'). 

La variedad de usos del valor 'NMOV' dificulta que se pueda emplear para identificar a quienes se han ido o han vuelto a España en el periodo. Esta información es imprescindible para imputar valores de otras columnas definidas posteriormente. 

In [ ]:
df['CONT1'].sort_values().unique()

In [ ]:
# Función cont_nmov: función auxiliar para facilitar la interpretación de respuesta 'NMOV' en 'CONTi' (i=1,2,...,7)
# Valores de todos los 'CONTi' cuando el i-ésimo es 'NMOV', agrupados por el i-ésimo: 
def cont_nmov(i):
    List=['CONT'+str(j) for j in range(i,8)]
    print (df[List][df['CONT'+str(i)]=='NMOV'].groupby('CONT'+str(i)).agg(['unique']))

In [ ]:
cont_nmov(1)
# 'CONT1'='NMOV' implica no movilidad en todo el periodo

In [ ]:
cont_nmov(2)
# 'CONT2'='NMOV' tiene un comportamiento distinto del resto de las variables 'CONTi'

In [ ]:
cont_nmov(3)
# 'CONT3'='NMOV' implica no movilidad anterior

In [ ]:
cont_nmov(4)
# 'CONT4'='NMOV' implica no movilidad anterior

In [ ]:
cont_nmov(5)
# 'CONT5'='NMOV' implica no movilidad anterior

In [ ]:
cont_nmov(6)
# 'CONT6'='NMOV' implica no movilidad anterior

#### 'ANOIPAIS1'
**Año de inicio del tiempo de residencia en el primer país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOIPAIS1'].sort_values().unique() # Blank as '   .'

In [ ]:
# Funcion re_cont: inspección e imputación de valores de variables condicionadas por respuesta 
# a 'CONTi' (i=1,2,...,7).
def re_cont(col,cont,blank,tipus='float'):
    print (df[[col,cont]].groupby(cont,dropna=False).agg(['unique']))
    df[col].replace({blank:None},inplace=True)
    df[col]=df[col].astype(tipus)
    print (df[[col,cont]].groupby(cont,dropna=False).agg(['unique']))
    return

In [ ]:
re_cont('ANOIPAIS1','CONT1',blank='   .')

#### 'ANOFPAIS1'
**Año final del tiempo de residencia en el primer país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOFPAIS1'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOFPAIS1','CONT1',blank='   .')

#### 'MESES1'
**Duración en meses** del tiempo de residencia en el **primer país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['MESES1'].sort_values().unique() # Blank as '  .'

In [ ]:
re_cont('MESES1','CONT1',blank='  .')

#### 'CONT2'
**Continente del segundo país de residencia** -en orden cronológico inverso- durante **al menos 3 meses en el periodo 2000-2009**: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NMOV'=Sin movilidad.

In [ ]:
df['CONT2'].sort_values().unique() 

#### 'ANOIPAIS2'
**Año de inicio del tiempo de residencia en el segundo país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOIPAIS2'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOIPAIS2','CONT2',blank='   .')
# Hay observaciones atípicas

In [ ]:
# Lista de columnas 'CONTi' (i=1,2,...,7) para simplificar notación: 
L_CONT=['CONT'+str(i) for i in range(1,8)]
L_CONT

In [ ]:
df[L_CONT+['ANOIPAIS2','ANOFPAIS2','MESES2']][(df['CONT2']=='NMOV')&(df['ANOIPAIS2'].notna())] 
# Observación peculiar: periodo intermedio registrado como 'sin movilidad' ('NMOV') con info temporal completa.

In [ ]:
# Corrección observación no homogénea: 
df.loc[3416,'CONT2']='EU27' #Se imputa este valor por entender que se trata de un periodo intermedio en España
df[['ANOIPAIS2','CONT2']].groupby('CONT2').agg(['unique'])

#### 'ANOFPAIS2'
**Año final del tiempo de residencia en el segundo país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOFPAIS2'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOFPAIS2','CONT2',blank='   .')
# Observaciones atipicas.

In [ ]:
df[L_CONT+['ANOIPAIS2','ANOFPAIS2','MESES2']][(df['CONT2']=='NMOV')&(df['ANOFPAIS2'].notna())] 
# Observación peculiar: Aparente no movilidad y con valores temporales incompletos (solo año final)

In [ ]:
# Corrección observación atípica: 
df.loc[733,'ANOFPAIS2']=None # Sin información suficiente para mejorar la interpretación de la respuesta. 
df[['ANOFPAIS2','CONT2']].groupby('CONT2').agg(['unique'])

#### 'MESES2'
**Duración en meses del tiempo de residencia en el segundo país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['MESES2'].sort_values().unique() # Blank as '  .'

In [ ]:
re_cont('MESES2','CONT2',blank='  .')

In [ ]:
# Revisión de las correcciones en 'CONT2' y las variables condicionadas por esta: 
cont_nmov(2)
# Tras la corrección en las observaciones atípicas, 'CONT2'='NMOV' implica no movilidad posterior.
# El resultado de las correcciones parece coherente con el resto de las variables 'CONTi'.

#### 'CONT3'
**Continente del tercer país de residencia** -en orden cronológico inverso- durante **al menos 3 meses en el periodo 2000-2009**: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NMOV'=Sin movilidad

In [ ]:
df['CONT3'].sort_values().unique() 

#### 'ANOIPAIS3'
**Año de inicio del tiempo de residencia en el tercer país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOIPAIS3'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOIPAIS3','CONT3',blank='   .')

#### 'ANOFPAIS3'
**Año final del tiempo de residencia en el tercer país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOFPAIS3'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOFPAIS3','CONT3',blank='   .')

#### 'MESES3'
**Duración en meses del tiempo de residencia en el tercer país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['MESES3'].sort_values().unique() # Blank as '  .'

In [ ]:
re_cont('MESES3','CONT3',blank='  .')

#### 'CONT4'
**Continente del cuarto país de residencia** -en orden cronológico inverso- durante **al menos 3 meses en el periodo 2000-2009**: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NMOV'=Sin movilidad.

In [ ]:
df['CONT4'].sort_values().unique() 

#### 'ANOIPAIS4'
**Año de inicio del tiempo de residencia en el cuarto país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOIPAIS4'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOIPAIS4','CONT4',blank='   .')

#### 'ANOFPAIS4'
**Año final del tiempo de residencia en el cuarto país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOFPAIS4'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOFPAIS4','CONT4',blank='   .')

#### 'MESES4'
**Duración en meses del tiempo de residencia en el cuarto país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['MESES4'].sort_values().unique() # Blank as '  .'

In [ ]:
re_cont('MESES4','CONT4',blank='  .')

#### 'CONT5'
**Continente del quinto país de residencia** -en orden cronológico inverso- durante **al menos 3 meses en el periodo 2000-2009**: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NMOV'=Sin movilidad.

In [ ]:
df['CONT5'].sort_values().unique() 

#### 'ANOIPAIS5'
**Año de inicio del tiempo de residencia en el quinto país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOIPAIS5'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOIPAIS5','CONT5',blank='   .')

#### 'ANOFPAIS5'
**Año final del tiempo de residencia en el quinto país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOFPAIS5'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOFPAIS5','CONT5',blank='   .')

#### 'MESES5'
**Duración en meses del tiempo de residencia en el quinto país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['MESES5'].sort_values().unique() # Blank as '  .'

In [ ]:
re_cont('MESES5','CONT5',blank='  .')

#### 'CONT6'
**Continente del sexto país de residencia** -en orden cronológico inverso- durante **al menos 3 meses en el periodo 2000-2009**: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NMOV'=Sin movilidad.

In [ ]:
df['CONT6'].sort_values().unique() 

#### 'ANOIPAIS6'
**Año de inicio del tiempo de residencia en el sexto país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOIPAIS6'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOIPAIS6','CONT6',blank='   .')

#### 'ANOFPAIS6'
**Año final del tiempo de residencia en el sexto país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOFPAIS6'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOFPAIS6','CONT6',blank='   .')

#### 'MESES6'
**Duración en meses del tiempo de residencia en el sexto país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['MESES6'].sort_values().unique() # Blank as '  .'

In [ ]:
re_cont('MESES6','CONT6',blank='  .')

#### 'CONT7'
**Continente del séptimo país de residencia** -en orden cronológico inverso- durante **al menos 3 meses en el periodo 2000-2009**: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NMOV'=Sin movilidad

In [ ]:
df['CONT7'].sort_values().unique() 

#### 'ANOIPAIS7'
**Año de inicio del tiempo de residencia en el séptimo país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOIPAIS7'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOIPAIS7','CONT7',blank='   .')

#### 'ANOFPAIS7'
**Año final del tiempo de residencia en el séptimo país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['ANOFPAIS7'].sort_values().unique() # Blank as '   .'

In [ ]:
re_cont('ANOFPAIS7','CONT7',blank='   .')

#### 'MESES7'
**Duración en meses del tiempo de residencia en el septimo país** mencionado como de residencia al menos 3 meses en el periodo 2000-2009. 

In [ ]:
df['MESES7'].sort_values().unique() # Blank as '  .'

In [ ]:
re_cont('MESES7','CONT7',blank='  .')

#### 'MOTFUERA1'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  

¿**Realizar el doctorado** fue un motivo para vivir fuera de España en el periodo 2000-2009? 1=Sí, blanco.

**Observaciones**: 
- Las variables con respuesta Sí o blanco se formulan solo a un subconjunto de la muestra. Para imputar valor No a quien fué preguntado y no respondió Sí (como distinto los valores blancos por no haber sido preguntados) es imprescindible identificar a quienes fueron preguntados; en este caso, quienes vivieron fuera de España al menos 3 meses durante el periodo de referencia.  
- El uso ambiguo del valor 'NMOV' en las variables 'CONTi' (i=1,2,...,7) hace preferible usar otras variables para identificar a los preguntados. Así, se asumirá que solo fueron preguntados por los motivos para irse de España los que identifican al menos uno de los motivos sugeridos (variables 'MOTFUERAi', i=1,2,...,7).

In [ ]:
df['MOTFUERA1'].sort_values().unique() # Blank as ' '

In [ ]:
# Variable auxiliar 'emigro': toma valor 1 para los encuestados que se identifican con al menos uno de los ...
# ... motivos sugeridos como razón para vivir fuera de España en el periodo referido. 
# Se imputa valor 0 (No) a los registros en blanco (provisional) y 1 (int) a '1' para habilitar operaciones: 
for i in range(1,8):
    df['MOTFUERA'+str(i)].replace({'1':1,' ':0},inplace=True) 
# Condición o filtro: 
cond_emigro=sum([df['MOTFUERA'+str(i)] for i in range(1,8)])!=0 # True si algún valor no nulo (algún Sí)
# Variable auxiliar: 
df['emigro']=0
df['emigro'].mask(cond_emigro,1,inplace=True)

In [ ]:
# Comprobación: 
df[['MOTFUERA'+str(i) for i in range(1,8)]+['emigro']].groupby('emigro').agg(['value_counts'])

In [ ]:
# Comprobación adicional: los no preguntados son los mismos para los que 'CONT1'='NMOV'.
df[['CONT1','emigro']][df['CONT1']=='NMOV'].groupby('emigro').agg(['value_counts'])

In [ ]:
# Función motfuera: inspeccion e imputación de valores de las variables 'MOTFUERAi' (i=1,2,...,7)
def motfuera(col):
    print(df[['emigro',col]].groupby('emigro').agg(['unique'])) 
    # Anteriormente se imputó No (0) a todas las no respuestas
    # Se corrige imputando None a quien no fue preguntado: 
    df[col].mask(df['emigro']==0,None,inplace=True)
    df[col]=df[col].astype('float')
    print(df[['emigro',col]].groupby('emigro').agg(['unique']))
    return

In [ ]:
motfuera('MOTFUERA1')

#### 'MOTFUERA2'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  

¿**Finalizar el postdoctorado o un contrato de trabajo** fue un motivo para vivir fuera de España en el periodo 2000-2009? 1=Sí, blanco.

In [ ]:
motfuera('MOTFUERA2')

#### 'MOTFUERA3'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  

¿Fueron **otros factores relacionados con el empleo o económicos** (traslado laboral porel mismo organismo o empresa, cambio de empleo, garantía u oferta de trabajo, o realización de un postdoctorado) motivos para vivir fuera de España en el periodo 2000-2009? 1=Sí, blanco.

In [ ]:
motfuera('MOTFUERA3')

#### 'MOTFUERA4'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  

¿Fueron **factores académicos** (mayor posibilidad de realizar publicaciones, desarrollo o continuidad de la tesis doctoral, trabajar en un área específica no existente en el país de origen, creación de un equipo propio o una nueva área de investigación) motivos para vivir fuera de España en el periodo 2000-2009? 1=Sí, blanco.

In [ ]:
motfuera('MOTFUERA4')

#### 'MOTFUERA5'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  

¿Fueron **factores personales o familiares** motivos para vivir fuera de España en el periodo 2000-2009? 1=Sí, blanco.

In [ ]:
motfuera('MOTFUERA5')

#### 'MOTFUERA6'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  

¿Fueron **factores políticos o administrativos** (finalización de permiso de residencia o visado, etc.) motivos para vivir fuera de España en el periodo 2000-2009? 1=Sí, blanco.

In [ ]:
motfuera('MOTFUERA6')

#### 'MOTFUERA7'
(Solo si se fue a vivir fuera de España en el periodo 2000-2009)  

¿Fueron **otros motivos** los que le llevaron a vivir fuera de España en el periodo 2000-2009? 1=Sí, blanco.

In [ ]:
motfuera('MOTFUERA7')

#### 'MOTVENIR1'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)  

¿Fue **finalizar el doctorado** un motivo para venir o volver a España en el periodo 2000-2009? 1=Sí, blanco.

**Observación**: Se procede de forma análoga a como se hizo con la familia de variables 'MOTFUERAi' para imputar valores perdidos debido a los itinerarios de la encuesta. 

In [ ]:
df['MOTVENIR1'].sort_values().unique() # Blank as ' '

In [ ]:
# Variable auxiliar 'regreso': toma valor 1 para los encuestados que se identifican con al menos uno de los ...
# ... motivos sugeridos como razón para venir o regresar a España en el periodo referido. 
# Se imputa valor 0 (No) a los registros en blanco (provisional) y 1 (int) a '1' para habilitar operaciones: 
for i in range(1,8):
    df['MOTVENIR'+str(i)].replace({'1':1,' ':0},inplace=True) 
# Condición o filtro: 
cond_regreso=sum([df['MOTVENIR'+str(i)] for i in range(1,8)])!=0 # True si algún valor no nulo (algún Sí)
# Variable auxiliar: 
df['regreso']=0
df['regreso'].mask(cond_regreso,1,inplace=True)

In [ ]:
# Inspección: 
df['regreso'].value_counts()

In [ ]:
# Comprobación: 
df[['MOTVENIR'+str(i) for i in range(1,8)]+['regreso']].groupby('regreso').agg(['value_counts'])

In [ ]:
# Inspección adicional I: 
df[L_CONT][df['regreso']==0].value_counts()

In [ ]:
# Inspección adicional II: 
df[L_CONT][df['regreso']==1].value_counts()
# Las inspecciones adicionales I y II permiten comprobar que el análisis de los valores de 'CONTi' no permite ...
# ... identificar a quienen vinieron o retornaron: algunas filas aparecen asociadas a 'regreso'=0 en algunos ...
# ... casos y a 'regreso'=1 en otros. 

In [ ]:
# Función motvenir: inspeccion e imputación de valores de las variables 'MOTVENIRi' (i=1,2,...,7)
def motvenir(col):
    print(df[['regreso',col]].groupby('regreso').agg(['unique']))
    # Anteriormente se imputó 0 (No) a todas las repuestas en blanco
    # Se imputa None a quien no fue preguntado: 
    df[col].mask(df['regreso']==0,None,inplace=True)
    df[col]=df[col].astype('float')
    print(df[['regreso',col]].groupby('regreso').agg(['unique']))
    return

In [ ]:
motvenir('MOTVENIR1')

#### 'MOTVENIR2'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   

¿Fueron **finalizar el postdoctorado o un contrato de trabajo** motivos para venir o volver a España en el periodo 2000-2009?

In [ ]:
motvenir('MOTVENIR2')

#### 'MOTVENIR3'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   

¿Fueron **otros factores relacionados con el empleo o económicos** (traslado laboral por el mismo organismo o empresa, cambio de empleo, garantía u oferta de trabajo, o realización de un postdoctorado) motivos para venir o volver a España en el periodo 2000-2009?

In [ ]:
motvenir('MOTVENIR3')

#### 'MOTVENIR4'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   

¿Fueron **factores académicos** (mayor posibilidad de realizar publicaciones, desarrollo o continuidad de la tesis doctoral, trabajar en un área específica no existente en el país de origen, creación de un equipo propio o una nueva área de investigación) motivos para venir o volver a España en el periodo 2000-2009?

In [ ]:
motvenir('MOTVENIR4')

#### 'MOTVENIR5'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   

¿Fueron **factores personales o familiares** motivos para venir o volver a España en el periodo 2000-2009?

In [ ]:
motvenir('MOTVENIR5')

#### 'MOTVENIR6'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   

¿Fueron **factores políticos o administrativos** (finalización de permiso de residencia o visado, etc.) motivos para venir o volver a España en el periodo 2000-2009?

In [ ]:
motvenir('MOTVENIR6')

#### 'MOTVENIR7'
(Solo si vino a vivir o regresó a España en el periodo 2000-2009)   

¿Fueron **otros motivos** los que le llevaron a venir o volver a España en el periodo 2000-2009?

In [ ]:
motvenir('MOTVENIR7')

#### 'PREVIRSE'
¿Tiene previsto marcharse a vivir fuera de España?  1=Sí, permanentemente, 2=Sí, temporalmente, 3= No. 

In [ ]:
df['PREVIRSE'].sort_values().unique() # No missing values

In [ ]:
df['PREVIRSE']=df['PREVIRSE'].astype('int') 
df['PREVIRSE'].sort_values().unique() 

#### 'MESANTESIR'
(Solo para respuesta 1 o 2 en variable 'PREVIRSE')  

Tiempo que considera que va a transcurrir antes de abandonar España:1=Menos de 6 meses, 2=De 6 a  12 meses, 3=De 1 a 2 años, 4=De 2 a 3 años, 5=De 3 a 4 años, 6=De 4 a 5 años, 7=De 5 a 10 años, 8=Más de 10 años.

In [ ]:
df['MESANTESIR'].sort_values().unique() # Blank as ' '

In [ ]:
df[['MESANTESIR','PREVIRSE']].groupby('PREVIRSE').agg(['unique']) # No missing values

In [ ]:
# Función re_previrse: inspección e imputación de valores de variables condicionadas por la respuesta a 'PREVIRSE'.
def re_previrse(col,blank=' ',tipus='float'):
    print (df[[col,'PREVIRSE']].groupby('PREVIRSE').agg(['unique']))
    df[col].replace({blank:None},inplace=True)
    df[col]=df[col].astype(tipus)
    print (df[[col,'PREVIRSE']].groupby('PREVIRSE').agg(['unique']))
    return    

In [ ]:
re_previrse('MESANTESIR')

#### 'MOTIRSE1'
(Solo para respuesta 1 o 2 en variable 'PREVIRSE')  

¿Fueron **finalizar el postdoctorado o un contrato de trabajo** motivos para tomar la decision de marcharse a vivir fuera de España? 1=Sí, blanco.

In [ ]:
df[['MOTIRSE1','PREVIRSE']].groupby('PREVIRSE').agg(['unique']) 

In [ ]:
# Función motirse: inspección e imputación de valores de la familia de variables 'MOTIRSEi' (i=1,2,...,6)
def motirse(col):
    print (df[[col,'PREVIRSE']].groupby('PREVIRSE').agg(['unique']))
    # Se imputa respuesta No (0) cuando no se ha contestado Sí (1) habiendo sido preguntado
    df[col].mask(((df['PREVIRSE']==1)|(df['PREVIRSE']==2))&(df[col]!='1'),0,inplace=True)
    # Se imputa None cuando no ha sido preguntado
    df[col].replace({' ':None},inplace=True)
    df[col]=df[col].astype('float')
    print (df[[col,'PREVIRSE']].groupby('PREVIRSE').agg(['unique']))
    return    

In [ ]:
motirse('MOTIRSE1')

#### 'MOTIRSE2'
(Solo para respuesta 1 o 2 en variable 'PREVIRSE')  

¿Fueron **otros factores relacionados con el empleo o económicos** (traslado laboral por el mismo organismo o empresa, cambio de empleo, garantía u oferta de trabajo, o realización de un postdoctorado motivos para tomar la decision de marcharse a vivir fuera de España? 1=Sí, blanco.

In [ ]:
motirse('MOTIRSE2')

#### 'MOTIRSE3'
(Solo para respuesta 1 o 2 en variable 'PREVIRSE')  

¿Fueron **factores académicos** (mayor posibilidad de realizar publicaciones, desarrollo o continuidad de la tesis doctoral, trabajar en un área específica no existente en el país de origen, creación de un equipo propio o una nueva área de investigación) motivos para tomar la decision de marcharse a vivir fuera de España? 1=Sí, blanco.

In [ ]:
motirse('MOTIRSE3')

#### 'MOTIRSE4'
(Solo para respuesta 1 o 2 en variable 'PREVIRSE')  

¿Fueron **factores personales o familiares** motivos para tomar la decision de marcharse a vivir fuera de España? 1=Sí, blanco.

In [ ]:
motirse('MOTIRSE4')

#### 'MOTIRSE5'
(Solo para respuesta 1 o 2 en variable 'PREVIRSE')  

¿Fueron **factores políticos o administrativos** (finalización de permiso de residencia o visado, etc.) motivos para tomar la decision de marcharse a vivir fuera de España? 1=Sí, blanco.

In [ ]:
motirse('MOTIRSE5')

#### 'MOTIRSE6'
(Solo para respuesta 1 o 2 en variable 'PREVIRSE')  

¿Fueron **otros motivos** las razones para tomar la decision de marcharse a vivir fuera de España? 1=Sí, blanco.

In [ ]:
motirse('MOTIRSE6')

#### 'CONTOUT'
(Solo para respuesta 1 o 2 en variable 'PREVIRSE')  

**Continente** del país **al que tiene pensado marcharse a vivir**: 'EU27'=Unión Europea, 'REUR'=Resto de Europa, 'EUCA'=EEUU y Canadá, 'RAME'=Resto de América, 'AFRI' = África, 'ASIA'=Asia, 'OCEA'=Oceanía, 'NMOV'=Sin movilidad.

In [ ]:
df['CONTOUT'].sort_values().unique() # No missing values

#### 'INV'
¿Estaba **trabajando como investigador** el 31 de diciembre de 2009? 1=Sí, 0=No. 

In [ ]:
df['INV'].sort_values().unique() # No missing values

In [ ]:
df['INV']=df['INV'].astype('int')
df['INV'].sort_values().unique() 

#### 'MOTNOINV1'
(Solo para respuesta No (0) en variable 'INV')  

¿Fue que **no estaba interesado** en investigar un motivo para no estar trabajando como investigador? 1=Sí, blanco.

In [ ]:
df['MOTNOINV1'].sort_values().unique() # Blank as ' '

In [ ]:
# Función motnoinv: inspección e imputación de valores de la familia de variables 'MOTNOINVi' (i=1,2,...,9)
def motnoinv(col):
    print (df[[col,'INV']].groupby('INV').agg(['unique']))
    # Se imputa respuesta No (0) cuando no se ha contestado Sí (1) habiendo sido preguntado
    df[col].mask((df['INV']==0)&(df[col]!='1'),0,inplace=True)
    # Se imputa None cuando no ha sido preguntado
    df[col].replace({' ':None},inplace=True)
    df[col]=df[col].astype('float')
    print (df[[col,'INV']].groupby('INV').agg(['unique']))
    return    

In [ ]:
motnoinv('MOTNOINV1')

#### 'MOTNOINV2'
(Solo para respuesta No (0) en variable 'INV')  

¿Fueron las **oportunidades laborales muy limitadas en el campo de la investigación** un motivo para no estar trabajando como investigador? 1=Sí, blanco.

In [ ]:
motnoinv('MOTNOINV2')

#### 'MOTNOINV3'
(Solo para respuesta No (0) en variable 'INV')  

¿Fue la **falta de estructura para el desarrollo de la carrera investigadora** un motivo para no estar trabajando como investigador? 1=Sí, blanco.

In [ ]:
motnoinv('MOTNOINV3')

#### 'MOTNOINV4'
(Solo para respuesta No (0) en variable 'INV')  

¿Fue la **baja remuneración** un motivo para no estar trabajando como investigador? 1=Sí, blanco.

In [ ]:
motnoinv('MOTNOINV4')

#### 'MOTNOINV5'
(Solo para respuesta No (0) en variable 'INV')  

¿Fueron las **malas condiciones laborales** un motivo para no estar trabajando como investigador? 1=Sí, blanco.

In [ ]:
motnoinv('MOTNOINV5')

#### 'MOTNOINV6'
(Solo para respuesta No (0) en variable 'INV')  

¿Fue la **falta de reconocimiento público a la investigación** un motivo para no estar trabajando como investigador? 1=Sí, blanco.

In [ ]:
motnoinv('MOTNOINV6')

#### 'MOTNOINV7'
(Solo para respuesta No (0) en variable 'INV')  

¿Fue la **incertidumbre sobre las perspectivas profesionales a largo plazo** un motivo para no estar trabajando como investigador? 1=Sí, blanco.

In [ ]:
motnoinv('MOTNOINV7')

#### 'MOTNOINV8'
(Solo para respuesta No (0) en variable 'INV')  

¿Fue la **jubilación** un motivo para no estar trabajando como investigador? 1=Sí, blanco.

In [ ]:
motnoinv('MOTNOINV8')

#### 'MOTNOINV9'
(Solo para respuesta No (0) en variable 'INV')  

¿Fueron **otros** los motivos para no estar trabajando como investigador? 1=Sí, blanco.

In [ ]:
motnoinv('MOTNOINV9')

#### 'ALGINV'
(Solo para respuesta No (0) en variable 'INV')  

¿Ha **trabajado alguna vez como investigador**? 1=Sí, 0=No.

In [ ]:
df[['ALGINV','INV']].groupby('INV').agg(['unique']) # No missing values

In [ ]:
# Se imputa valor 1 a quienes trabajaban como investigadores en el momento de la encuesta (1 en 'INV')
df['ALGINV'].replace({' ':1},inplace=True)
df['ALGINV']=df['ALGINV'].astype('int')
df[['ALGINV','INV']].groupby('INV').agg(['unique'])

#### 'MOTINV1'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')  

¿Fue el **trabajo creativo e innovador** un motivo para decidir trabajar como investigador? 1=Sí, blanco

In [ ]:
df[['MOTINV1','ALGINV']].groupby('ALGINV').agg(['unique']) 

In [ ]:
# Función motinv: inspección e imputación de valores de la familia de variables 'MOTINVi' (i=1,2,...,6)
def motinv(col):
    print(df[[col,'ALGINV']].groupby('ALGINV').agg(['unique']))
    # Se imputa respuesta No (0) a quienes han trabajado como investigadores y no responden Sí (1)
    df[col].mask((df['ALGINV']==1)&(df[col]!='1'),0,inplace=True)
    # Se imputa None cuando no ha sido preguntado
    df[col].replace({' ':None},inplace=True)
    df[col]=df[col].astype('float')
    print (df[[col,'ALGINV']].groupby('ALGINV').agg(['unique']))
    return    

In [ ]:
motinv('MOTINV1')

#### 'MOTINV2'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')  

¿Fue la **alta remuneración** un motivo para decidir trabajar como investigador? 1=Sí, blanco

In [ ]:
motinv('MOTINV2')

#### 'MOTINV3'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')  

¿Fue la **promoción profesional** un motivo para decidir trabajar como investigador? 1=Sí, blanco

In [ ]:
motinv('MOTINV3')

#### 'MOTINV4'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')  

¿Fue la **seguridad laboral** un motivo para decidir trabajar como investigador? 1=Sí, blanco

In [ ]:
motinv('MOTINV4')

#### 'MOTINV5'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')  

¿Fueron las **buenas condiciones laborales** un motivo para decidir trabajar como investigador? 1=Sí, blanco

In [ ]:
motinv('MOTINV5')

#### 'MOTINV6'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')   

¿Fue la **contribución a la sociedad** un motivo para decidir trabajar como investigador? 1=Sí, blanco

In [ ]:
motinv('MOTINV6')

#### 'MOTINV7'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')  

¿Fue que **no pudo elegir otro empleo** un motivo para decidir trabajar como investigador? 1=Sí, blanco

In [ ]:
motinv('MOTINV7')

#### 'MOTINV8'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')   

¿Fue el mero **interés por la investigación y el conocimiento** un motivo para decidir trabajar como investigador? 1=Sí, blanco

In [ ]:
motinv('MOTINV8')

#### 'MOTINV9'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')    

¿Fue **otros** los motivos para decidir trabajar como investigador? 1=Sí, blanco

In [ ]:
motinv('MOTINV9')

#### 'MESESINV'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')    

Numero total de **meses que ha trabajado como investigador** a lo largo de su vida. 

In [ ]:
df[['MESESINV','ALGINV']].groupby('ALGINV').agg(['unique']) # Blank as '  .'. No missing values

In [ ]:
# Función re_alginv: inspección e imputación de valores de variables condicionadas por respuesta a 'ALGINV'.
def re_alginv(col,blank):
    print (df[[col,'ALGINV']].groupby('ALGINV').agg(['unique']))
    # Se imputa valor 0 a quienes no han trabajado nunca como investigadores: 
    df[col].replace({blank:0},inplace=True) # No missing values para varlor 1 de 'ALGINV'
    df[col]=df[col].astype('int')
    print (df[[col,'ALGINV']].groupby('ALGINV').agg(['unique']))

In [ ]:
re_alginv('MESESINV',blank='  .')

#### 'ARTIC'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')    

Número de **artículos** que le han sido publicados o aceptados por una publicación especializada entre enero de 2007 y diciembre de 2009.

In [ ]:
df[['ARTIC','ALGINV']].groupby('ALGINV').agg(['unique']) # Blank as '  .'. No missing values

In [ ]:
# Se imputa valor 0 a quienes no han trabajado nunca como investigadores: 
re_alginv('ARTIC',blank='  .')

#### 'LIBROS'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')    

Número de **libros o monografías** (incluyendo colaboraciones) que le han sido publicados o aceptados por una publicación especializada entre enero de 2007 y diciembre de 2009.

In [ ]:
df[['LIBROS','ALGINV']].groupby('ALGINV').agg(['unique']) # Blank as '  .'. No missing values

In [ ]:
# Se imputa valor 0 a quienes no han trabajado nunca como investigadores: 
re_alginv('LIBROS',blank='  .')

#### 'NUMPAT'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')    

Número de **solicitudes de patentes** en las que figuraba como inventor entre enero de 2007 y diciembre de 2009

In [ ]:
df[['NUMPAT','ALGINV']].groupby('ALGINV').agg(['unique']) # Blank as ' .'. No missing values

In [ ]:
# Se imputa valor 0 a quienes no han trabajado nunca como investigadores: 
re_alginv('NUMPAT',blank=' .')

#### 'PATNAC'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')    

Número de **solicitudes de patentes con protección nacional** en las que figuraba como inventor entre enero de 2007 y diciembre de 2009

In [ ]:
df[['PATNAC','ALGINV']].groupby('ALGINV').agg(['unique'])  # Blank as ' .'. No missing values

In [ ]:
# Se imputa valor 0 a quienes no han trabajado nunca como investigadores: 
re_alginv('PATNAC',blank=' .')

#### 'PATEU'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')    

Número de **solicitudes de patentes con protección europea** en las que figuraba como inventor entre enero de 2007 y diciembre de 2009

In [ ]:
df[['PATEU','ALGINV']].groupby('ALGINV').agg(['unique']) # Blank as ' .'. No missing values

In [ ]:
# Se imputa valor 0 a quienes no han trabajado nunca como investigadores: 
re_alginv('PATEU',blank=' .')

#### 'PATCOOP'
(Solo para respuesta Sí (1) en la variable 'ALGINV' o Sí (1) a la variable 'INV')    

Número de **solicitudes de patentes con protección del Tratado de Cooperación de Patentes** en las que figuraba como inventor entre enero de 2007 y diciembre de 2009

In [ ]:
df[['PATCOOP','ALGINV']].groupby('ALGINV').agg(['unique']) # Blank as ' .'. No missing values

In [ ]:
# Se imputa valor 0 a quienes no han trabajado nunca como investigadores: 
re_alginv('PATCOOP',blank=' .')

#### 'PATCOOP'   
Número de sus **patentes de productos o procesos que han sido comercializadas** o han obtenido ya las licencias necesarias para ello entre enero de 2007 y diciembre de 2009.

In [ ]:
df[['PATCOM','ALGINV']].groupby('ALGINV').agg(['unique']) # No missing values

In [ ]:
df['PATCOM']=df['PATCOM'].astype('int')

#### 'COMPA'   
¿Ha **constituido una empresa** entre enero de 2007 y diciembre de 2009? 1=Sí, 0=No.

In [ ]:
df['COMPA'].sort_values().unique() # No missing values

In [ ]:
df['COMPA']=df['COMPA'].astype('int')

#### 'TUTOR'   
¿Ha **dirigido algún Master o tesis doctoral** entre enero de 2007 y diciembre de 2009? 1=Sí, 0=No.

In [ ]:
df['TUTOR'].sort_values().unique() # No missing values

In [ ]:
df['TUTOR']=df['TUTOR'].astype('int')

#### 'COOPERA'   
¿Ha **cooperado con grupos de investigación extranjeros** entre enero de 2007 y diciembre de 2009? 1=Sí, 0=No.

In [ ]:
df['COOPERA'].sort_values().unique() # No missing values

In [ ]:
df['COOPERA']=df['TUTOR'].astype('int')

#### 'INVFUT'   
¿Tiene **intención de dedicarse a la investigación** en el periodo 2010-2011? 1=Sí, 0=No.

In [ ]:
df['INVFUT'].sort_values().unique() # No missing values

In [ ]:
df['INVFUT']=df['INVFUT'].astype('int')

#### 'FACTOR'   
Variable no descrita en la documentación adjunta. 

In [ ]:
df['FACTOR'].sort_values().unique()

In [ ]:
for col in df.columns: 
    print (col, df[col].dtype)

In [ ]:
df.to_csv('/home/bea/Escritorio/KSchool_TFM/Principales/Encuesta_RRHH_Ciencia_Tecnologia_2009/prep_doct.csv')